# Проект "Создание рекомендательной системы для интернет-магазина"
## Группа: DSPRML - 109   
## Выполнил: Новиков Павел
## Задача: Повысить прибыль от допродаж в интернет-магазине на 20 %
## Метрика: Precision@3  
Данная метрика выбрана потому что у компании клиента только три места для показа товаров на главной странице.  
**Precision@3** это точность рекомендательной системы рассчитанная с учётом подмножества рекомендаций от ранга 1 до 3.  

\begin{align}
        Precision = \frac{TP}{TP+FP} , где
    \end{align}

TP - Количество релевантных рекомендаций  
TP + FP - Общее количество рекомендованных элементов
## Этапы выполнения:
| Таск | Подзадача | Спринт |
|--------|---------|---------|
|Постановка задачи|Уточнение бизнес-постановки задачи|1|
||Выбор технических метрик качества|1|
||Получение данных|1|
|Исследование данных|Описание структуры данных|1|
||Статистический анализ данных|1|
|Создание факторов для модели|Генерация факторов, связанных с айтемами|2|
||Генерация факторов айтем-юзер|2|
|Проведение экспериментов|Коллаборативная фильтрация|3|
||Факторизационные машины|3|
||XGBoost для задачи классификации|3|
|Создание MVP|Проектирование API сервиса|4|
||Создание веб-сервиса с моделью|4|
|Контейнеризация|Создание Docker-контейнера с сервисом|4|


# Environment

In [1]:
! pip install scipy -q
! pip install lightfm -q
! pip install optuna -q
! pip install scikit-surprise -q
! pip install tqdm -q
! pip install xgboost -q

In [2]:
import gc
import random

import pandas as pd
import numpy as np

import scipy
import optuna

from tqdm import tqdm
import collections
from collections import Counter

from datetime import datetime
from scipy.sparse import csr_matrix
from surprise.model_selection import train_test_split as sur_train_test_split

# Импортируем необходимые нам компоненты и считаем данные с помощью специального метода Reader:
from surprise import Dataset as surDataset
from surprise import Reader
from surprise import KNNBasic, KNNWithMeans
from collections import defaultdict


from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from sklearn.model_selection import train_test_split
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb

N_TRIAL = 10

# Загрузка данных

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/FeatureGenerationNotebook (1).ipynb
/kaggle/input/item_properties_part1.csv/item_properties_part1.csv
/kaggle/input/item_properties_part2.csv/item_properties_part2.csv
/kaggle/input/events.csv/events.csv


# Данные
Для решения задачи нам понадобятся 2 таблицы:
- таблица 'item-user', содержащая совершенные взаимодействия пользователей и товаров
- таблица свойств товаров

### Таблица **item-user**

## Файл events
events — датасет с событиями:
- timestamp — время события
- visitorid — идентификатор пользователя
- event — тип события
- itemid — идентификатор объекта
- transactionid — идентификатор транзакции, если она проходила

In [4]:
# загрузим данные о покупках и иных взаимодействиях items и users
events = pd.read_csv('/kaggle/input/events.csv/events.csv')
# s1 = events[events['event'] == 'transaction'].sample(n=5000, random_state = 42)
s1 = events[events['event'] == 'transaction']
n = int(s1.shape[0]/2)
s2 = events[events['event'] == 'addtocart'].sample(n=n, random_state = 42)
s3 = events[events['event'] == 'view'].sample(n=n, random_state = 42)
events_c = pd.concat([s1, s2, s3], ignore_index=False)

# переведем формат времени к обычному виду
events['event_datetime'] = pd.to_datetime(events['timestamp'], unit='ms', origin='unix')
# отсортируем данные от самых ранних записей до поздних
events = events.sort_values(['event_datetime']).reset_index(drop=True)
events.head(5)

,timestamp,visitorid,event,itemid,transactionid,event_datetime
0,1430622004384,693516,addtocart,297662,NaN,2015-05-03 03:00:04.384
1,1430622011289,829044,view,60987,NaN,2015-05-03 03:00:11.289
2,1430622013048,652699,view,252860,NaN,2015-05-03 03:00:13.048
3,1430622024154,1125936,view,33661,NaN,2015-05-03 03:00:24.154
4,1430622026228,693516,view,297662,NaN,2015-05-03 03:00:26.228


Посмотрим сколько пропусков в признаках

In [5]:
#Количество пропусков
events.isna().sum()

timestamp               0
visitorid               0
event                   0
itemid                  0
transactionid     2733644
event_datetime          0
dtype: int64

In [6]:
print(f'Количество дубликатов до удаления: {len(events)- len(events.drop_duplicates())}')
events.drop_duplicates(inplace=True)
print(f'Количество дубликатов после удаления: {len(events)- len(events.drop_duplicates())}')

Количество дубликатов до удаления: 460
Количество дубликатов после удаления: 0


In [7]:
events['event'].value_counts()

event
view           2664218
addtocart        68966
transaction      22457
Name: count, dtype: int64

В колонке *view* описаны действия, которые совершали *visitiorid* на сайте:
- view (просмотр карточки товара)
- addtocart (добавление в корзину)
- transaction (покупка)

Наблюдается характерная для индустрии продаж "воронка"

In [8]:
print(f'Количество users: {len(set(events.visitorid))}')
mask1 = events[events['event'] == 'transaction']
print(f'Количество users, сделавших покупку items: {len(set(mask1.visitorid))}')
print(f'Количество items: {len(set(events.itemid))}')
print(f'Количество купленных items: {len(set(mask1.itemid))}')
mask2 = mask1[['visitorid','itemid']].groupby(['visitorid','itemid'], as_index = False).count()
print(f'Количество пар users-items, среди совершивших покупку: {mask2.shape[0]}')

Количество users: 1407580
Количество users, сделавших покупку items: 11719
Количество items: 235061
Количество купленных items: 12025
Количество пар users-items, среди совершивших покупку: 21270


In [9]:
mask3 = mask1[['visitorid','itemid']].groupby(['itemid'], as_index = False).count().sort_values(by='visitorid', ascending = False)[:10]
print(f'Больше всего было куплено следующих items:')
print(f'{mask3.reset_index(drop=True)}')

Больше всего было куплено следующих items:
   itemid  visitorid
0  461686        133
1  119736         97
2  213834         92
3  312728         46
4    7943         46
5  445351         45
6   48030         41
7  420960         38
8  248455         38
9   17478         37


In [10]:
transaction_nan = set(events[events['transactionid'] != events['transactionid']]['visitorid'])
transaction_true = set(events[events['transactionid'] == events['transactionid']]['visitorid'])
print(f'Количество users не совершивших покупку: {len(transaction_nan.difference(transaction_true))}')

Количество users не совершивших покупку: 1395861


Для дальнейшего анализа нам понадобится случайный user, не совершавший покупок ранее.

In [11]:
list(transaction_nan.difference(transaction_true))[42]

42

In [12]:
events[events['visitorid'] == int(42)]

,timestamp,visitorid,event,itemid,transactionid,event_datetime
1793162,1437968765299,42,view,60399,NaN,2015-07-27 03:46:05.299


Не совершал.

In [13]:
events[events['visitorid'] == int(1173192)]

,timestamp,visitorid,event,itemid,transactionid,event_datetime
1298811,1436151386829,1173192,view,7943,NaN,2015-07-06 02:56:26.829
1298998,1436151969319,1173192,addtocart,7943,NaN,2015-07-06 03:06:09.319
1299080,1436152241199,1173192,transaction,7943,6530.0,2015-07-06 03:10:41.199


А этот покупал.

# Моделирование

# Surprise
Коллаборативная фильтрация  

Поочередно применим следующие подходы:  
- memory-based в модификации user-based, основанный на близости пользователей,
- memory-based в модификации item-based, основанной на близости продуктов.

## User-based, Item-based

**Коллаборативная фильтрация на основе близости пользователей(user-based)** — это метод рекомендаций, который предпологает, что пользователь будет заинтересован в тех продуктах, которые высоко оценили или купили другие пользователи с аналогичными предпочтениями или поведением.  
**Коллаборативная фильтрация на основе близости продуктов(item-based)** — это метод рекомендаций, который предполагает, что пользователь, который купил или оценил один продукт, скорее всего будет заинтересован в продуктах, которые были куплены или оценены другими пользователями, совершившими аналогичные покупки или оценки.

Подготовим данные для использования библиотеки Surprise.  


In [14]:
#Исходя из важности каждого действия составим индекс популярности товаров, придав разные веса действиям посетителей сайта.
weigths = {
    'view': 0,
    'addtocart': 1,
    'transaction': 2
}
# добавим данные о рейтинге items
events['weigths'] = events['event'].apply(lambda x: weigths[x])
# отфильтруем данные о просмотрах как малоинформативные, мы решаем задачу о продажах, а не о просмотрах
events_sur = events[events['weigths'] !=0].reset_index(drop=True)
# events_sur = events.copy()

events_sur = events_sur[["visitorid", "itemid", "weigths"]].astype({'visitorid':'int32','itemid':'int32','weigths':'int8'}).rename(columns =
    {'visitorid': 'userid', 'itemid': 'itemid', 'weigths':'ratings'})


In [15]:
events_sur.isna().sum()

userid     0
itemid     0
ratings    0
dtype: int64

In [16]:
events_sur.head(3)

,userid,itemid,ratings
0,693516,297662,1
1,693516,297662,1
2,979664,338222,1


Для построения рекомендательной системы на основе memory-based подхода будем использовать библиотеку Surprise.  
Cоздадим объект Reader из библиотеки Surprise.  
Параметр rating_scale определяет шкалу оценок, используемую в данных.   
В данном случае, оценки варьируются от 1 до 2 включительно.  
Логика работы данной библиотеки не подразумевает обязательную предварительную группировку данных, поэтому, мы отправляем в построение датасета и дальнейшую разбивку данные о взаимодействии как есть.

In [17]:
reader = Reader(rating_scale=(1, 2))

# The columns must correspond to user id, item id and ratings (in that order).
data = surDataset.load_from_df(events_sur, reader)

### Train/test split

In [18]:
trainset, testset = sur_train_test_split(data, test_size=0.3, random_state=13, shuffle=False)
len(testset)

27427

In [19]:
len_trainset = events_sur.shape[0] - len(testset)
len_trainset

63996

Посмотрим на характер представления данных в trainset

In [20]:
for index in range(5):
    user_id = trainset.to_raw_uid(trainset.all_users()[index])
    item_id = trainset.to_raw_iid(trainset.all_items()[index])
    rating = trainset.ur[index]  # Оценка пользователя для данного товара
    print("Пользователь {}, Изделие {}, Рейтинг {}".format(user_id, item_id, rating))

Пользователь 693516, Изделие 297662, Рейтинг [(0, 1.0), (0, 1.0)]
Пользователь 979664, Изделие 338222, Рейтинг [(1, 1.0)]
Пользователь 260113, Изделие 125751, Рейтинг [(2, 1.0)]
Пользователь 319455, Изделие 342530, Рейтинг [(3, 1.0)]
Пользователь 345781, Изделие 438400, Рейтинг [(4, 1.0), (4, 2.0)]


In [21]:
trainset.all_users()

range(0, 27191)

In [22]:
for index in range(27191):
    user_id = trainset.to_raw_uid(trainset.all_users()[index])
    if int(user_id) == int(1173192):
        user_id = trainset.to_raw_uid(trainset.all_users()[index])
        item_id = trainset.to_raw_iid(trainset.all_items()[index])
        rating = trainset.ur[index]  # Оценка пользователя для данного товара
        print("Пользователь {}, Изделие {}, Рейтинг {}".format(user_id, item_id, rating))

Пользователь 1173192, Изделие 284609, Рейтинг [(46, 1.0), (46, 2.0)]


Взятый для примера пользователь попал в trainset

In [23]:
algo_users = KNNWithMeans(k=15, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo_items = KNNWithMeans(k=15, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo_users.fit(trainset)
algo_items.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [24]:
def precision_k(predictions,k=3,threshold=1.5):
    """Возвращает метрику precision at k для каждого user"""

    # Создадим предсказания для каждого user
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

    return precisions


predictions_users = algo_users.test(testset)
predictions_items = algo_items.test(testset)
precisions_users = precision_k(predictions_users, k=3, threshold=1.5)
precisions_items = precision_k(predictions_items, k=3, threshold=1.5)

# Precision and recall can then be averaged over all users
print(f'Precision@3 users-based: {round((sum(prec for prec in precisions_users.values())/len(precisions_users)),3)}')
print(f'Precision@3 items-based: {round((sum(prec for prec in precisions_items.values())/len(precisions_items)),3)}')

Precision@3 users-based: 0.002
Precision@3 items-based: 0.002


Посмотрим на пример предсказания

In [25]:
# Каков прогнозируемый рейтинг для пользователя с userid = 1173192
uid = int(1173192)
iid = int(7943)
print(f'Предсказание на основе близости items, для item раннее уже покупавшимися user: ')
algo_items.predict(uid, iid, verbose=True)
print(f'Предсказание на основе близости users,с учетом items раннее уже покупавшимися user: ')
algo_users.predict(uid, iid, verbose=True)

Предсказание на основе близости items, для item раннее уже покупавшимися user: 
user: 1173192    item: 7943       r_ui = None   est = 1.50   {'actual_k': 2, 'was_impossible': False}
Предсказание на основе близости users,с учетом items раннее уже покупавшимися user: 
user: 1173192    item: 7943       r_ui = None   est = 1.50   {'actual_k': 15, 'was_impossible': False}


Prediction(uid=1173192, iid=7943, r_ui=None, est=1.4991326699307206, details={'actual_k': 15, 'was_impossible': False})

In [26]:
# Каков прогнозируемый рейтинг для пользователя с userid = 1173192
uid = int(1173192)
iid = int(284609)
print(f'Предсказание на основе близости items, для item раннее не покупавшимися user: ')
algo_items.predict(uid, iid, verbose=True)
print(f'Предсказание на основе близости users, с учетом items раннее не покупавшимися user: ')
algo_users.predict(uid, iid, verbose=True)

Предсказание на основе близости items, для item раннее не покупавшимися user: 
user: 1173192    item: 284609     r_ui = None   est = 1.00   {'actual_k': 0, 'was_impossible': False}
Предсказание на основе близости users, с учетом items раннее не покупавшимися user: 
user: 1173192    item: 284609     r_ui = None   est = 1.50   {'actual_k': 0, 'was_impossible': False}


Prediction(uid=1173192, iid=284609, r_ui=None, est=1.5, details={'actual_k': 0, 'was_impossible': False})

In [27]:
# Уникальный идентификатор пользователя для которого нам нужны рекомендации
uid = int(1173192)  # пример user_id который ранее делал покупки

# Получение всех возможных item_ids из набора данных
items = trainset.all_items()
iid_list = [trainset.to_raw_iid(iid) for iid in items]

# Получение прогнозов для всех item_ids
predictions = [algo_items.predict(uid, iid) for iid in iid_list]

# Сортировка прогнозов по оценке
top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

# Получение топ-5 рекомендаций
top_5_recommendations = top_predictions[:5]

# Вывод результатов
for pred in top_5_recommendations:
    print(f"User {pred.uid} может купить item {pred.iid} с предсказанным rating {pred.est:.2f}")

User 1173192 может купить item 29877 с предсказанным rating 2.00
User 1173192 может купить item 158834 с предсказанным rating 2.00
User 1173192 может купить item 309748 с предсказанным rating 2.00
User 1173192 может купить item 239596 с предсказанным rating 2.00
User 1173192 может купить item 79442 с предсказанным rating 2.00


In [28]:
# Уникальный идентификатор пользователя для которого нам нужны рекомендации
uid = int(1173192)  # пример user_id который ранее делал покупки

# Получение всех возможных item_ids из набора данных
items = trainset.all_items()
iid_list = [trainset.to_raw_iid(iid) for iid in items]

# Список items, по которым еще нет оценок от данного user
user_rated_items = {iid for (uid, iid) in trainset.ur[trainset.to_inner_uid(uid)]}
recommend_items = [iid for iid in iid_list if iid not in user_rated_items]

# Получение предсказаний для всех items, которые еще не оценены user
predictions = [algo_items.predict(uid, iid) for iid in recommend_items]

# Отбор топ-5 рекомендаций на основе предсказанных оценок
top_5_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:5]

# Вывод результатов
for pred in top_5_recommendations:
    print(f"User {pred.uid} может купить ранее не приобретенный item {pred.iid} с предсказанным rating {pred.est:.2f}")

User 1173192 может купить ранее не приобретенный item 29877 с предсказанным rating 2.00
User 1173192 может купить ранее не приобретенный item 158834 с предсказанным rating 2.00
User 1173192 может купить ранее не приобретенный item 309748 с предсказанным rating 2.00
User 1173192 может купить ранее не приобретенный item 239596 с предсказанным rating 2.00
User 1173192 может купить ранее не приобретенный item 79442 с предсказанным rating 2.00


In [29]:
# Уникальный идентификатор пользователя для которого нам нужны рекомендации
uid = int(42)  # пример user_id которого нет в trainset
# Получение всех возможных item_ids из набора данных
items = trainset.all_items()
iid_list = [trainset.to_raw_iid(iid) for iid in items]

# Список items, по которым еще нет оценок от данного user
try:
    user_rated_items = {iid for (uid, iid) in trainset.ur[trainset.to_inner_uid(uid)]}
except ValueError:
    print('Данный user ещё не взаимодействовал с items')
recommend_items = [iid for iid in iid_list if iid not in user_rated_items]

# Получение предсказаний для всех items, которые еще не оценены user
predictions = [algo_items.predict(uid, iid) for iid in recommend_items]

# Отбор топ-5 рекомендаций на основе предсказанных оценок
top_5_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:5]

# Вывод результатов
for pred in top_5_recommendations:
    print(f"User {pred.uid} может приобрести item {pred.iid} с предсказанным rating {pred.est:.2f}")

Данный user ещё не взаимодействовал с items
User 42 может приобрести item 297662 с предсказанным rating 1.25
User 42 может приобрести item 338222 с предсказанным rating 1.25
User 42 может приобрести item 125751 с предсказанным rating 1.25
User 42 может приобрести item 342530 с предсказанным rating 1.25
User 42 может приобрести item 438400 с предсказанным rating 1.25


In [30]:
del trainset
del testset
del data
del precisions_users
del precisions_items
del predictions_users
del predictions_items
del algo_items
del algo_users
gc.collect()

0

# LightFM
Коллаборативная фильтрация + матричная факторизация

In [31]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2755641 entries, 0 to 2756100
Data columns (total 7 columns):
 #   Column          Dtype         
---  ------          -----         
 0   timestamp       int64         
 1   visitorid       int64         
 2   event           object        
 3   itemid          int64         
 4   transactionid   float64       
 5   event_datetime  datetime64[ns]
 6   weigths         int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 168.2+ MB


## train/test split

In [32]:
# s1 = events[events['event'] == 'transaction'].sample(n=5000, random_state = 42)
s1 = events[events['event'] == 'transaction']
n = int(s1.shape[0]/2)
s2 = events[events['event'] == 'addtocart'].sample(n=n, random_state = 42)
s3 = events[events['event'] == 'view'].sample(n=n, random_state = 42)
events_c = pd.concat([s1, s2, s3], ignore_index=False)
events_c = events_c[["visitorid", "itemid", "weigths"]].astype({'visitorid':'int32','itemid':'int32','weigths':'int8'}).rename(columns =
    {'visitorid': 'userid', 'itemid': 'itemid', 'weigths':'weights'})

train_orders, test_orders = train_test_split(events_c, test_size=0.3, shuffle=False, random_state=42)

# Сформируем train
def create_train_data(dataset):
    data = dataset.groupby(['userid','itemid'], as_index=False)['weights'].agg('sum')
#     data["weights"] = np.where(data["weights"]>0, 1, data["weights"]) # cap it at 5 ограничить на 5
    return data
train = create_train_data(train_orders)

# Сформируем test
def create_test_data(test, train):
    data = test.groupby(['userid','itemid'], as_index=False).agg('count')
    data = data[['userid','itemid']]
    data = data.merge(train["userid"].drop_duplicates()) # remove users not in training data
    data = data.merge(train["itemid"].drop_duplicates()) # remove items not training data
    return data
test = create_test_data(test_orders, train)

In [33]:
del s1
del s2
del s3
# del events_c
gc.collect()

0

LightFM может создавать предсказания для новых/холодных товаров и пользователей. Мы посмотрим, как это сделать, а пока удалим из Test все предметы/пользователей, которых нет в Train.

Проверим есть ли в тесте users, которых нет в train

In [34]:
print(len(set(test_orders['itemid']).difference(set(train_orders['itemid']))))
print(len(set(test['itemid']).difference(set(train['itemid']))))
print(len(set(test_orders['userid']).difference(set(train_orders['userid']))))
print(len(set(test['userid']).difference(set(train['userid']))))

6993
0
11231
0


Таких users и items в train/test нет

Обычно мы хотим найти новые товары, с которыми пользователи еще не взаимодействовали, чтобы рекомендовать их.   
Однако для такой сферы, как розничная торговля продуктами питания, рекомендация товаров, которые пользователи уже покупали, вероятно, является законным, хотя и не очень оригинальным вариантом.  
Давайте создадим тестовый набор, включающий повторно купленные товары, и набор, содержащий только новые для пользователя товары, т. е. не купленные ранее.  
Затем мы сможем сравнить, как работает наша модель на обоих наборах.

In [35]:
# Создадим тестовый набор, исключающий повторные покупки
def create_new_only_test_data(test, train):
    # test = test.drop(columns=['weights'])
    data = test.merge(train,  how='left', left_on=['userid','itemid'], right_on = ['userid','itemid'])
    data = data[data['weights'].isna()]
    data = data.drop(columns=['weights'])
    return data
test_new = create_new_only_test_data(test, train)

После подготовки признаков users и items начнем эксперименты по моделированию с целью выявить конфигурацию данных и гиперпараметров дающих наибольшую метрику lightFM

In [36]:
# Уникальный список идентификаторов пользователей
train_users = train["userid"].unique()

# Уникальный список идентификаторов товаров
train_items = train["itemid"].unique()

LightFM  сопоставляет каждый элемент, пользователя и все различные характеристики с уникальными индексами, которые затем используются для поиска этих пользователей, элементов и характеристик в различных матрицах, используемых для создания рекомендаций.

In [37]:
# Create user, item and feature mappings: (user id map, user feature map, item id map, item feature map)
dataset = Dataset() # helper function
dataset.fit(train_users, # creates mappings between userIDs and row indices for LightFM
                 train_items)
len(dataset.mapping()) # we always get 4x mappings out

4

Несмотря на то, что в этот раз мы не передавали список характеристик пользователя или предмета, мы все равно получаем в ответ 4 отображения (идентификатор пользователя, характеристики пользователя, идентификатор предмета, характеристики предмета).  
Это происходит потому, что LightFM создает отдельные матрицы для пользователей и предметов и связанных с ними характеристик.   
В нашем случае матрицы характеристик - это просто идентификационные характеристики, т. е. каждый пользователь/предмет - это собственная характеристика, поэтому мы получаем отображения характеристик, хотя передали только пользователей/предметы.    
Отображения идентификатора и характеристики будут одинаковой длины, но позже, когда мы начнем добавлять дополнительные характеристики, мы увидим, что отображения характеристик станут длиннее.

In [38]:
# Соответствия пользователей и предметов
user_mappings = dataset.mapping()[0]
item_mappings = dataset.mapping()[2]

len(user_mappings), len(item_mappings)

(16541, 14422)

Если мы посмотрим на mappings, то увидим, как наши идентификаторы пользователей были отображены на непрерывные индексы, начинающиеся с 0.   
Например, идентификатор пользователя 4 отображается на индекс 0, идентификатор пользователя 17 - на индекс 1 и т. д.   
LightFM использует эти внутренние отображения для построения матриц взаимодействия, а также матриц поиска пользователей и предметов в их представлениях.

In [39]:
# Have a look at the mappings
all_users = list(user_mappings.keys())
print(all_users[:5]) # the first 5 user IDs
list(user_mappings.items())[:5] # first 5 mappings

[172, 186, 264, 416, 419]


[(172, 0), (186, 1), (264, 2), (416, 3), (419, 4)]

Помимо создания сопоставлений с внутренними индексами LightFM, создадим обратные сопоставления, чтобы иметь возможность вернуться к реальным пользователям и продуктам, когда мы будем составлять рекомендации.

In [40]:
# Create inverse mappings
inv_user_mappings = {v:k for k, v in user_mappings.items()}
inv_item_mappings = {v:k for k, v in item_mappings.items()}
list(inv_item_mappings.items())[:5]

[(0, 10034), (1, 465522), (2, 49029), (3, 161949), (4, 459835)]

После того как мы получили наши отображения между пользователями и предметами и их индексами LightFM, мы можем использовать функцию build_interactions() для создания матрицы взаимодействий пользователя и предмета, на основе которой LightFM будет учиться.   
Функция принимает итерируемый список, например список, кортеж, словарь и т. д. взаимодействий пользователей и предметов, а также необязательный столбец веса, который говорит LightFM, насколько более или менее важным является конкретное взаимодействие при обучении.   
Если мы не передадим весовой столбец, LightFM по умолчанию присвоит каждому взаимодействию значение 1. Давайте создадим нашу матрицу взаимодействий:

In [41]:
# Создайте матрицу взаимодействий для каждого пользователя, предмета и веса
train_interactions, train_weights = dataset.build_interactions(train[['userid', 'itemid', 'weights']].values)
train_interactions, train_weights

(<16541x14422 sparse matrix of type '<class 'numpy.int32'>'
 	with 27324 stored elements in COOrdinate format>,
 <16541x14422 sparse matrix of type '<class 'numpy.float32'>'
 	with 27324 stored elements in COOrdinate format>)

Мы видим, что в результате получаем две разреженные матрицы.   
Первая - это матрица взаимодействий, которая записывает взаимодействие пользователя и предмета и представляет собой 1 строку для пользователя и 1 столбец для предмета, где 1 отмечено, что взаимодействие имело место.  
Другая разреженная матрица - это матрица весов.   
Если бы мы не передавали веса в функцию, она была бы идентична нашей матрице взаимодействий, но поскольку мы использовали веса, эта матрица имеет ту же форму, но записывает индивидуальные веса для взаимодействий.   
Давайте преобразуем их в плотные типы и посмотрим, чем они отличаются.

In [42]:
# Have a look at the matrices
# train_interactions.todense(), train_weights.todense() # weights and interactions are the same if we just use 1s
# веса и взаимодействия одинаковы, если мы используем только 1s

Таким образом, мы видим, что обе матрицы фиксируют взаимодействие пользователя с элементом.   
Теперь давайте создадим матрицы взаимодействий для наших тестовых данных.   
Нам не нужно беспокоиться о весах для них, поскольку они используются только в обучении, но LightFM все равно создает матрицу весов по умолчанию.

In [43]:
191554 in all_users

False

In [44]:
# Create Test set - notice that LightFM automatically makes it the same size as Train to preserve integer mappings
test_interactions, test_weights = dataset.build_interactions(test[['userid', 'itemid']].values)
test_interactions, test_weights

# Create a new-products-purchased-only Test set
test_new_interactions, test_new_weights = dataset.build_interactions(test_new[['userid', 'itemid']].values)
test_new_interactions, test_new_weights

(<16541x14422 sparse matrix of type '<class 'numpy.int32'>'
 	with 585 stored elements in COOrdinate format>,
 <16541x14422 sparse matrix of type '<class 'numpy.float32'>'
 	with 585 stored elements in COOrdinate format>)

Если мы посмотрим на отображения, то увидим, как наши идентификаторы пользователей были отображены на непрерывные индексы, начинающиеся с 0.   
Например, идентификатор пользователя 4 отображается на индекс 0, идентификатор пользователя 17 - на индекс 1 и т. д.   
LightFM использует эти внутренние отображения для построения матриц взаимодействия, а также матриц поиска пользователей и предметов в их представлениях.

## Baseline
Теперь у нас есть матрицы взаимодействия, и мы можем создать нашу первую модель.
  
Для начала мы просто используем матричную факторизации без каких-либо дополнительных возможностей.   
Сначала мы определим нашу модель, вызвав LightFM() и установив различные параметры.   
Я практически оставил их по умолчанию, но все равно написал их, чтобы дать представление о том, какие у нас есть возможности.  
После определения модели мы вызовем функцию fit() для обучения модели и передадим ей нашу матрицу взаимодействий.   
По умолчанию модель обучается только в течение 1 эпохи,я установил значение 20.

In [45]:
model = LightFM(no_components=10,  # the dimensionality of the feature latent embeddings
                			learning_schedule='adagrad', # тип используемого алгоритма оптимизации
                			loss='warp', # loss type
                			learning_rate=0.05, # set the initial learning rate
                			item_alpha=0.0, # L2 penalty on item features
                			user_alpha=0.0, # L2 penalty on users features
                			max_sampled=10, # maximum number of negative samples used during WARP fitting
                			random_state=123)

model.fit(train_interactions, # our training data
               epochs = 20,
               verbose=True)

Epoch: 100%|██████████| 20/20 [00:00<00:00, 38.10it/s]


Оценка рекомендателей несколько отличается от обычных задач регрессии/классификации.
Мы используем бинарные данные «взаимодействовал/не взаимодействовал» в качестве нашей цели, так что это похоже на проблему классификации.   
Однако большинство пользователей не взаимодействуют с большинством предметов, поэтому такая мера, как точность, не подходит, поскольку мы, вероятно, получим точность 99 %+, просто предсказав 0 для всех.   
У нас также могут быть пользователи, которые с высокой вероятностью купят много items, но также будут и те, кто вряд ли купит что-либо вообще.   
Для обычной классификации мы бы хотели предсказать 0 для пользователей, которые вряд ли что-то купят. Однако в рекомендательной системе это невозможно. Если у нас есть 10 мест для рекомендаций на веб-странице, которые нужно заполнить для каждого пользователя, мы не можем просто оставить их пустыми, потому что считаем, что они вряд ли что-то купят, нам все равно нужно им что-то показать.

Лучший способ представить себе проблему рекомендаций - это проблема ранжирования, т. е. нам нужно показать 3 items каждому user, поэтому мы хотим выбрать 3 лучших товаров для этого пользователя, даже если мы считаем, что вероятность того, что он купит какой-либо товар, невелика.  
Таким образом, вместо традиционных показателей классификации, таких как Precision или Recall, нам нужно адаптировать их с учетом того факта, что мы должны выдать определенное количество рекомендаций для каждого пользователя.   
Обычно для этого используется «метрика @ k», где «k» - это количество слотов, которые нам нужно заполнить, или рекомендаций, которые мы должны вывести на поверхность для каждого пользователя.   
В LightFM есть несколько различных встроенных метрик оценки, и мы будем использовать Precision@3, то есть среднее значение Precision по пользователям для 3 рекомендаций с наивысшим рейтингом для каждого пользователя.   
Мы можем рассчитать их для наших наборов данных Train, Test и Test-with-only-new-items:

In [46]:
# Measure how well it did in the Test period
for metric in [precision_at_k]:
    # Get the precision and recall for Train and Test
    for data, name in [(train_interactions, "Train"), (test_interactions, "Test ")]:
        print(f"{name} {metric.__name__}: %.2f" %
              metric(model,
                     data,
                     k=3).mean())

    # What about for just new-to-user purchases?
    print(f"Test new {metric.__name__}: %.3f" %
          metric(model,
                 test_new_interactions,
                 train_interactions=train_interactions, # supress previously bought prods from being recommended
                 k=3).mean())

Train precision_at_k: 0.05
Test  precision_at_k: 0.02
Test new precision_at_k: 0.002


Итак, наша первая модель получила precision@3 в 0.06 на обучающих данных, но она значительно снижается до 0.03 на тестовых данных и еще больше - до 0,006 для предсказания того, какие новые товары пользователи могут купить.   

Чтобы использовать функцию predict() в LightFM, нам нужно передать ей список идентификаторов пользователей и идентификаторов товаров в несколько идиосинкразическом формате. Согласно документации, "если вы хотите сгенерировать оценку для нескольких предметов (например, [7, 8, 9]) для двух пользователей (например, [0, 1]), правильным способом вызова этого метода будет использование lfm.predict([0, 0, 0, 1, 1, 1], [7, 8, 9, 7, 8, 9]), а _не_ lfm.predict([0, 1], [7, 8, 9]), как вы можете изначально ожидать". Таким образом, по сути, нам нужно повторяющееся значение User ID для сопоставления с каждым ID элемента, для которого мы хотим получить предсказание. Чтобы получить все предсказания для всех пользователей сразу, мы можем построить некоторый список, прежде чем передавать их в predict.



In [47]:
# Создадим матрицу всех пользователей и элементов, чтобы получить для них прогнозы
n_users, n_items = train_interactions.shape

# Используем lightFM to create predictions for all users and all items
scoring_user_ids = np.concatenate([np.full((n_items, ), i) for i in range(n_users)]) # повторим user ID для всех проб
scoring_item_ids = np.concatenate([np.arange(n_items) for i in range(n_users)]) # повторим весь диапазон идентификаторов item IDs x количество user
scores = model.predict(user_ids = scoring_user_ids,
                                     item_ids = scoring_item_ids)
scores = scores.reshape(-1, n_items) # получим одну строку на каждого user
recommendations = pd.DataFrame(scores)
recommendations.shape

# Посмотрим на predicted scores for the first 5 users and first 5 items
recommendations.iloc[:5,:5]

,0,1,2,3,4
0,1.038077,1.483925,-1.259877,-2.377848,-1.074042
1,-0.876130,-0.251737,0.886304,-0.868349,0.169893
2,-2.068120,-1.051147,-0.772391,0.035885,1.127448
3,-0.864871,0.352245,-0.655463,-1.011093,0.020541
4,-0.979257,-0.039548,-0.308408,-0.987077,0.189624


В приведенном выше результате у нас есть 1 строка для ID пользователя и 1 столбец для ID элемента в порядке их индексов отображения LightFM, например, индекс LightFM пользователя 0 - это наша первая строка, а индекс LightFM ID элемента 0 - это наш первый столбец.   
Фактические оценки предсказаний не имеют смысла, кроме как в качестве средства создания рейтингов, т. е. результаты предсказаний не являются вероятностями и не сравнимы между пользователями.

Мы также можем извлечь эмбеддинги для пользователей и элементов непосредственно из модели и вычислить предсказания вручную. В LightFM есть функция get_representations(), которая берет на себя умножение различных эмбеддингов признаков, связанных с user/item, на их соответствующие веса, чтобы создать окончательное представление, которое мы затем можем извлечь. Окончательное предсказание - это просто точечное произведение между эмбеддингими пользователя и предмета с соответствующими bias.
Bias, как правило, играет роль кодирования популярности предмета, что позволяет эмбеддингим, как предполагается, отразить основную природу пользователя или предмета.

In [48]:
# Загрузим скрытые эмбеддинги, чтобы попробовать вычислить предсказания вручную
item_biases, item_embeddings = model.get_item_representations()
user_biases, user_embeddings = model.get_user_representations()


In [49]:
print(f'item_biases: {item_biases.shape}')

print(f'item_embeddings: {item_embeddings.shape}')

print(f'user_biases: {user_biases.shape}')

print(f'user_embeddings: {user_embeddings.shape}')

user_embeddings[:3]

item_biases: (14422,)
item_embeddings: (14422, 10)
user_biases: (16541,)
user_embeddings: (16541, 10)


array([[-0.5487363 ,  0.4600652 ,  0.47471553, -0.11965575, -0.41323468,
        -0.465555  ,  0.58405864,  0.5039226 ,  0.4581329 , -0.18938358],
       [ 0.24005225, -0.24752939,  0.58221847,  0.53106725,  0.28505597,
        -0.00156947, -0.22094288,  0.4399231 , -0.18007647,  0.5352441 ],
       [ 0.45207486, -0.49556842,  0.22783557, -0.07038024,  0.51493824,
        -0.21341527, -0.542651  , -0.34276217, -0.35513148, -0.30472428]],
      dtype=float32)

Преобразуем рекомендации в нечто более понятное, извлекая 10 лучших рекомендованных товаров для каждого пользователя.
Приведем 10 лучших рекомендаций с учетом и без учета предыдущих покупок, а также сами предыдущие покупки, чтобы мы могли увидеть, хорошо ли наши новые рекомендации соответствуют историческим предпочтениям.

In [50]:
# 10 лучших прогнозов для каждого пользователя
k=10
top_10 = np.argsort(-scores, axis=1) [::, :k]

# Получим информацию о предыдущих покупках для каждого user
previous = np.array(train_interactions.todense())

# Получим предыдущие покупки и лучшие прогнозы для user 1173192
user = user_mappings.get(1173192)

print("Предыдущие покупки:", *[inv_item_mappings.get(key) for key in np.array(range(previous.shape[1]))[previous[user]>0]], sep="\n")
print("Top 10 recommendations:", *sorted(zip([inv_item_mappings.get(key) for key in top_10[user]], range(k)), key = lambda x: x[1]), sep="\n")

Предыдущие покупки:
7943
Top 10 recommendations:
(461686, 0)
(7943, 1)
(320130, 2)
(119736, 3)
(29196, 4)
(9877, 5)
(213834, 6)
(268755, 7)
(12217, 8)
(318333, 9)


Мы видим, что наш пользователь ранее приобрел 7943, и LightFM повторно рекомендовал бы этот itemid первой десятке при составлении прогноза.  
Уберем из рекомендаций все ранее купленные itemid и посмотрим, как они изменятся.
Один из простых способов сделать это - использовать нашу матрицу тренировочных взаимодействий, в которой все ранее приобретенные items записаны как 1, умножить ее на большое число, а затем просто вычесть его из наших оценок, чтобы искусственно снизить оценки для всех ранее приобретенных линий.

In [51]:
# Удалим ранее купленные items из прогнозов
top_10_new  = np.argsort(-(scores-(previous*999999)), axis=1)[::, :k] # вычтем предыдущие покупки из прогнозов

# Получим top predictions for user 1173192
print("Предыдущие покупки:", *[inv_item_mappings.get(key) for key in np.array(range(previous.shape[1]))[previous[user]>0]], sep="\n")
print("Top 10 new recommendations:", *sorted(zip([inv_item_mappings.get(key) for key in top_10_new[user]], range(k)), key = lambda x: x[1]), sep="\n")

Предыдущие покупки:
7943
Top 10 new recommendations:
(461686, 0)
(320130, 1)
(119736, 2)
(29196, 3)
(9877, 4)
(213834, 5)
(268755, 6)
(12217, 7)
(318333, 8)
(48030, 9)


In [52]:
print(f'Больше всего было куплено следующих items:')
print(f'{mask3.reset_index(drop=True)}')

Больше всего было куплено следующих items:
   itemid  visitorid
0  461686        133
1  119736         97
2  213834         92
3  312728         46
4    7943         46
5  445351         45
6   48030         41
7  420960         38
8  248455         38
9   17478         37


На этот раз мы получаем список рекомендаций, совершенно новых для пользователя.  
Мы видим, что item = 461686, которое раньше было первым в списке, теперь занимает второе место.  
Мы видим, что некоторые из других самых продаваемых item, например 461686 и 119736, также попали в список.   
Обнаружить, что модель в итоге рекомендует самые продаваемые вина, можно довольно часто.
Хотя это не обязательно плохо, но если мы хотим попробовать рекомендовать более необычные или менее популярные товары, есть быстрое решение, которое мы можем попробовать с помощью LightFM.

Наряду с репрезентацией пользователей и предметов модель также изучает предубеждения пользователей и предметов.   
Обычно они заключаются в том, чтобы определить, насколько популярен тот или иной предмет, и затем повысить его оценку в окончательном прогнозе.
Чтобы делать предсказания без понятия популярности, мы можем просто повторить наше точечное произведение без предубеждений:

In [53]:
without_biases = (model.user_embeddings @ model.item_embeddings.T)
without_biases

top_10_without_biases = np.argsort(-without_biases, axis=1) [::, :k]
print("Топ 10 популярных рекомендаций без bias:", *sorted(zip([inv_item_mappings.get(key) for key in top_10_without_biases[user]], range(k)), key = lambda x: x[1]), sep="\n")

Топ 10 популярных рекомендаций без bias:
(358845, 0)
(444968, 1)
(170283, 2)
(376151, 3)
(405714, 4)
(318785, 5)
(439770, 6)
(427541, 7)
(72715, 8)
(211975, 9)


Хотя 7943 по-прежнему находится в списке, остальные рекомендации выглядят гораздо более непонятными.

Это был user, который совершал покупки ранее. Посмотрим, что выдаст алгоритм для пользователя, не совершавшего ранее покупок

In [54]:
try:
    user = user_mappings.get(42)
    print("Предыдущие покупки:", *[inv_item_mappings.get(key) for key in np.array(range(previous.shape[1]))[previous[user]>0]], sep="\n")
    print("Top 10 recommendations:", *sorted(zip([inv_item_mappings.get(key) for key in top_10[user]], range(k)), key = lambda x: x[1]), sep="\n")
except:
    print('Построить прогноз по данному user не представляется возможным')

Построить прогноз по данному user не представляется возможным


## Вычисление сходства между элементами
Поскольку мы уже извлекли наши представления элементов (из модели with-biases) для ручного создания прогнозов, мы можем использовать их для поиска сходства между элементами.   
Для этого мы используем косинусное сходство:

In [55]:
# Find similar items
item_to_item = pd.DataFrame(cosine_similarity(item_embeddings))
item_to_item.index = item_mappings.keys()
item_to_item.columns = item_mappings.keys()

# Find other products that should be similar to these top sellers
for i in ['461686', "7943", "358895", "48030"]:
    print(item_to_item[int(i)].sort_values(ascending=False)[:5])

461686    1.000000
82608     0.949055
327712    0.870809
22493     0.865629
112766    0.857698
Name: 461686, dtype: float32
7943      1.000000
62043     0.913152
279457    0.896531
213954    0.895326
66691     0.880064
Name: 7943, dtype: float32
358895    1.000000
98101     0.916210
420731    0.895590
435864    0.880757
294713    0.868241
Name: 358895, dtype: float32
48030     1.000000
432607    0.886573
277247    0.869574
168111    0.846971
164863    0.842246
Name: 48030, dtype: float32


Т.к. мы не знаем фактически какие товары зашифрованы, мы не можем понять насколько данные связи между items интуитивны.
Настроим гиперпараметры нашей модели и посмотрим, сможем ли мы получить дополнительную производительность.


## Настройка гиперпараметров с помощью Optuna


Мы воспользуемся пакетом Optuna, который попытается автоматически найти оптимальный набор гиперпараметров из нашего пространства поиска. Для этого он проводит многократные испытания и моделирует производительность LightFM как функцию различных гиперпараметров и значений, которые мы дали Optuna для использования.


Чтобы использовать Optuna, мы сначала создаем "исследование", которое представляет собой пространство поиска гиперпараметров, наборы данных, которые мы хотим использовать, и нашу метрику оценки, которую мы затем возвращаем.  
Чтобы избежать повторного использования тестовых данных, мы разделим Train на меньший набор Train и Validation с помощью функции LightFM train_test_split().  
Следует отметить, что при случайном разбиении данных не сохраняется хронология покупок, как в случае с нашими реальными данными Train-Test.   
Также существует вероятность того, что, поскольку наши данные настолько разрежены, у нас могут быть пользователи, все взаимодействия которых попадают либо в набор Train, либо в Validation.   
Повторные покупки невозможны, поэтому для целей настройки наши прогоны будут максимально похожи на Train и Test-new.

In [56]:
# Define our hyperparameter seearch space
def objective(trial):

    # Use LightFMs inbuilt train-test split function to create train and validation subsets
    train, val = random_train_test_split(train_interactions, test_percentage=0.25, random_state=42)

    # Define the hyperparameter space
    param = {
        'no_components': trial.suggest_int("no_components", 5, 64),
        "learning_schedule": trial.suggest_categorical("learning_schedule", ["adagrad", "adadelta"]),
        "loss":  trial.suggest_categorical("loss", ["bpr", "warp", "warp-kos"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 1),
        "item_alpha": trial.suggest_float("item_alpha", 1e-10, 1e-06, log=True),
        "user_alpha": trial.suggest_float("user_alpha", 1e-10, 1e-06, log=True),
        "max_sampled": trial.suggest_int("max_sampled", 5, 15),
    }
    epochs = trial.suggest_int("epochs", 20, 50)

    model_base = LightFM(**param, random_state=123)
    model_base.fit(train,
              epochs = epochs,
              verbose=True)

    val_precision = precision_at_k(model_base,
                                   val,
                                   train_interactions=train,
                                   k=3).mean()

    return val_precision

# Define the study
study = optuna.create_study(direction="maximize")

[I 2024-06-29 07:33:37,514] A new study created in memory with name: no-name-c02028a8-f1c5-4e1b-9457-c0b7121f7015


Еще одна замечательная особенность Optuna - мы можем передать ей исходные значения гиперпараметров, чтобы дать ей "теплый старт" в плане значений для исследования и базовой производительности, которую она должна побить при выполнении оптимизации.  
Хотя мы не использовали никакой регуляризации в исходной модели, чтобы сохранить параметры в том же логарифмическом масштабе, что и пробные значения, мы дадим им самый минимум.

Еще одна приятная особенность Optuna заключается в том, что сохраняются лучшие параметры из всех просмотренных на данный момент исследований, так что если мы прерываем его, то не теряем все наработки до этого момента.   
По окончании исследования мы можем распечатать лучшие значения гиперпараметров.

In [59]:
# Add in our original hyperparmeter values as a starting point for Optuna
study.enqueue_trial(params={"no_components":10,
                            					"learning_schedule":'adagrad',
                            					"loss":'warp',
                            					"learning_rate":0.05,
                            					"item_alpha":1e-10,
                            					"user_alpha":1e-10,
                            					"max_sampled":10,
                            					"epochs":20})

# Run the optimisation
study.optimize(objective, n_trials=N_TRIAL*3)

best_params = study.best_params
for k, v in best_params.items():
    print(k,":",v)

Epoch: 100%|██████████| 20/20 [00:00<00:00, 49.62it/s]
[I 2024-06-29 07:44:51,724] Trial 30 finished with value: 0.004002171102911234 and parameters: {'no_components': 10, 'learning_schedule': 'adagrad', 'loss': 'warp', 'learning_rate': 0.05, 'item_alpha': 1e-10, 'user_alpha': 1e-10, 'max_sampled': 10, 'epochs': 20}. Best is trial 20 with value: 0.005494505632668734.
Epoch: 100%|██████████| 45/45 [00:04<00:00, 10.30it/s]
[I 2024-06-29 07:45:08,686] Trial 31 finished with value: 0.005494505632668734 and parameters: {'no_components': 60, 'learning_schedule': 'adadelta', 'loss': 'warp-kos', 'learning_rate': 0.9130635088366955, 'item_alpha': 8.320205926652867e-08, 'user_alpha': 1.6673542005498293e-07, 'max_sampled': 8, 'epochs': 45}. Best is trial 20 with value: 0.005494505632668734.
Epoch: 100%|██████████| 41/41 [00:04<00:00, 10.11it/s]
[I 2024-06-29 07:45:24,660] Trial 32 finished with value: 0.004884005058556795 and parameters: {'no_components': 57, 'learning_schedule': 'adadelta', 'los

no_components : 53
learning_schedule : adadelta
loss : warp
learning_rate : 0.826297867169715
item_alpha : 2.7834720549771643e-07
user_alpha : 2.8829209631989213e-07
max_sampled : 7
epochs : 44


Мы видим, что лучшая модель имела довольно высокое количество компонентов (число измерений в эмбеддингах) и довольно низкий уровень регуляризации.   
В Optuna есть функция, которая пытается измерить, насколько важен каждый гиперпараметр с точки зрения вклада в итоговую производительность модели.  
Она использует случайный лес и значения гиперпараметров на каждой итерации, чтобы попытаться предсказать производительность пробной модели для этой итерации.   
Посмотрим, какие гиперпараметры, по мнению Optuna, оказали большее влияние на итоговую производительность нашей модели.

### Важность гиперпараметров

In [60]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

Таким образом, похоже, что наибольшее значение для оптимизации функции потери имеет **learning_schedule**, а на втором месте находится **no_components**.   
Параметр **learning_schedule**  

Этот параметр определяет расписание обучения, то есть алгоритм, который используется для обновления весов модели во время обучения.   
LightFM поддерживает несколько вариантов:
**adagrad:** Адаптивный градиентный спуск, который подстраивает скорость обучения для каждого параметра.
**adadelta:** Принимает во внимание более длинную историю градиентов, которые используются для регулировки скоростей обучения.

Параметр **no_components**

Этот параметр определяет количество латентных факторов, используемых в модели.   
Латентные(скрытые) факторы — это абстрактные характеристики, которые модель пытается выявить из данных, чтобы описать пользователей и элементы (например, продукты, фильмы и т.д.) в терминах меньшего числа признаков.

Чем больше количество латентных факторов, тем более детально модель может представлять пользователей и элементы.  
Однако увеличение этого числа может привести к переобучению (overfitting), особенно если данных недостаточно для того, чтобы обоснованно оценить все факторы. Малое количество факторов, наоборот, может не захватить достаточное количество информации из данных (underfitting).   

Теперь обучим финальную модель на 100% Train и посмотрим, как она работает на наших тестовых данных.

In [61]:
# Tidy up epochs as not a parameter to be passed to LightFM() directly
num_epochs = best_params['epochs'] # save best epochs as a separate object
del best_params['epochs'] # then remove it from best_params object

# Train with the best parameters
model_base = LightFM(**best_params, random_state=123)

model_base.fit(train_interactions,
          epochs = num_epochs,
          verbose=True)

# Measure how well it did in the Test period
for metric in [precision_at_k]:
    # Get the precision and recall for Train and Test
    for data, name in [(train_interactions, "Train"), (test_interactions, "Test ")]:
        print(f"{name} {metric.__name__}: %.2f" %
              metric(model_base,
                     data,
                     k=3).mean())

    # What about for just new-to-user purchases?
    print(f"Test new {metric.__name__}: %.3f" %
          metric(model_base,
                 test_new_interactions,
                 train_interactions=train_interactions, # supress previously bought prods from being recommended
                 k=3).mean())

Epoch: 100%|██████████| 44/44 [00:04<00:00, 10.42it/s]


Train precision_at_k: 0.39
Test  precision_at_k: 0.19
Test new precision_at_k: 0.004


Наша настроенная модель показывает незначительное улучшение по сравнению с данными Test-new, так что, похоже, она оказалась успешной.  
Если бы мы захотели, мы могли бы провести больше испытаний в надежде, что производительность продолжит улучшаться.   
Давайте извлечем из новой модели эмбеддинги users и items и посмотрим, стали ли наши похожие предметы более осмысленными.

## Добавление weigths
В самом начале мы создали некоторые весовые коэффициенты между пользователем и предметом, чтобы отразить, что пользователи покупают некоторые предметы чаще, чем другие.   
В нашей первоначальной модели все взаимодействия рассматривались одинаково, но теперь давайте попробуем запустить ее с весами.   
Помимо увеличения веса более важных взаимодействий, мы могли бы уменьшить вес менее важных.   
Это один из способов, который предлагается для работы с очень популярными товарами, чтобы не рекомендовать их всегда и сделать наши рекомендации более разнообразными.

Чтобы использовать весовые коэффициенты в функции train_test_split(), нам нужно передать их отдельно вместе с тем же random_state, чтобы убедиться, что разбиения происходят в одном и том же месте.    
Затем мы можем передать в Optuna дополнительный гиперпараметр "использовать или не использовать веса", чтобы посмотреть, обнаружит ли она какую-либо пользу от их включения.

In [62]:
def objective(trial):

    train, val = random_train_test_split(train_interactions, test_percentage=0.25, random_state=42)
    train_weights, val_weight = random_train_test_split(train_interactions, test_percentage=0.25, random_state=42)

    param = {
        'no_components': trial.suggest_int("no_components", 5, 64),
        "learning_schedule": trial.suggest_categorical("learning_schedule", ["adagrad", "adadelta"]),
        "loss":  trial.suggest_categorical("loss", ["warp"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 1),
        "item_alpha": trial.suggest_float("item_alpha", 1e-10, 1e-06, log=True),
        "user_alpha": trial.suggest_float("user_alpha", 1e-10, 1e-06, log=True),
        "max_sampled": trial.suggest_int("max_sampled", 5, 15),
    }
    epochs = trial.suggest_int("epochs", 20, 50)
    sample_weights = trial.suggest_categorical("sample_weight", ["None", "train_weights"]) # add weights as a parameter

    model = LightFM(**param, random_state=123)
    model.fit(train,
              sample_weight=eval(sample_weights),
              epochs = epochs,
              verbose=True)

    val_precision = precision_at_k(model,
                                   val,
                                   train_interactions=train,
                                   k=3).mean()

    return val_precision

study = optuna.create_study(direction="maximize")

# Add in our original hyperparmeter values as a starting point for Optuna
best_params["epochs"]=num_epochs # manually add epochs
best_params["sample_weight"] ="None" # add in the fact the previous models didn't use weights
best_params["loss"] ="warp" #can't use kos with weights so switch it to warp
study.enqueue_trial(best_params)

study.optimize(objective, n_trials=N_TRIAL)

best_params = study.best_params
for k, v in best_params.items():
    print(k,":",v)

[I 2024-06-29 07:54:45,349] A new study created in memory with name: no-name-f89d538c-6be6-4c40-94e0-d2c921a993f0
Epoch: 100%|██████████| 44/44 [00:03<00:00, 13.78it/s]
[I 2024-06-29 07:54:59,635] Trial 0 finished with value: 0.0067155067808926105 and parameters: {'no_components': 53, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.826297867169715, 'item_alpha': 2.7834720549771643e-07, 'user_alpha': 2.8829209631989213e-07, 'max_sampled': 7, 'epochs': 44, 'sample_weight': 'None'}. Best is trial 0 with value: 0.0067155067808926105.
Epoch: 100%|██████████| 28/28 [00:01<00:00, 17.77it/s]
[I 2024-06-29 07:55:06,429] Trial 1 finished with value: 0.004137837793678045 and parameters: {'no_components': 23, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.797799841633417, 'item_alpha': 1.6474986734312985e-08, 'user_alpha': 6.428767241784689e-08, 'max_sampled': 7, 'epochs': 28, 'sample_weight': 'None'}. Best is trial 0 with value: 0.0067155067808926105.
Epoc

no_components : 53
learning_schedule : adadelta
loss : warp
learning_rate : 0.826297867169715
item_alpha : 2.7834720549771643e-07
user_alpha : 2.8829209631989213e-07
max_sampled : 7
epochs : 44
sample_weight : None


Похоже, Optuna обнаружила, что отказ от использования весов взаимодействия (или, по крайней мере, тех, которые мы создали в самом начале) не улучшил производительность. Это, наверное, неудивительно, учитывая, что мы повышали вес всех товаров, регулярно покупаемых пользователями, но, похоже, большинство пользователей покупают всего несколько вин, так что разница всегда была незначительной. Для полноты картины давайте обучим нашу новую модель на Train и посмотрим, как она работает.

In [63]:
num_epochs = best_params['epochs']
sample_weights=best_params['sample_weight']

del best_params['epochs']
del best_params['sample_weight']

# Train with the best parameters
model_w = LightFM(**best_params, random_state=123)

model_w.fit(train_interactions,
          sample_weight=eval(sample_weights),
          epochs = num_epochs,
          verbose=True)

# Measure how well it did in the Test period
for metric in [precision_at_k]:
    # Get the precision and recall for Train and Test
    for data, name in [(train_interactions, "Train"), (test_interactions, "Test ")]:
        print(f"{name} {metric.__name__}: %.2f" %
              metric(model_w,
                     data,
                     k=3).mean())

    # What about for just new-to-user purchases?
    print(f"Test new {metric.__name__}: %.3f" %
          metric(model_w,
                 test_new_interactions,
                 train_interactions=train_interactions, # supress previously bought prods from being recommended
                 k=3).mean())

Epoch: 100%|██████████| 44/44 [00:04<00:00, 10.03it/s]


Train precision_at_k: 0.39
Test  precision_at_k: 0.19
Test new precision_at_k: 0.004


## LightFM with items feature

По-прежнему производительность находится примерно на уровне 4% для Test precision at 3.   
Может быть, мы попробуем добавить несколько дополнительных функций элементов, чтобы попытаться увеличить производительность.  
Применим последовательно 2 подхода к item's feature engineering

### Items feature 1 version

Построение модели рекомендаций с использованием признаков items без их подробного описания.  

Эти более обобщенные признаки могут быть хорошими и могут лучше работать в сценариях с холодными стартовыми данными.

In [64]:
# загрузим справочник товаров
item_1 = pd.read_csv('/kaggle/input/item_properties_part1.csv/item_properties_part1.csv')
item_2 = pd.read_csv('/kaggle/input/item_properties_part2.csv/item_properties_part2.csv')
properties = pd.concat([item_1, item_2])
properties.loc[:,'value'] = properties['value'].str.replace('.','',regex=False)
properties.info()
del item_1
del item_2
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Index: 20275902 entries, 0 to 9275902
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  int64 
 1   itemid     int64 
 2   property   object
 3   value      object
dtypes: int64(2), object(2)
memory usage: 773.5+ MB


17

Нам нужно найти наиболее полезные свойства items.   
А они есть у фактически купленных items.   
По этому их и купили, очевидно.   
Поэтому отфильтруем справочник всех товаров.  
Оставим фактически купленные items.  

In [65]:
events_deal=train_orders[train_orders['weights'] == 2].groupby(['itemid'])['weights'].agg('count').sort_values(ascending = False)
events_deal[:5]

itemid
461686    133
119736     97
213834     92
312728     46
7943       46
Name: weights, dtype: int64

Это список наиболее привлекательных items, следовательно свойства этих items наиболее желанны для потребителя.

In [66]:
# Возьмем только самые продаваемые proprties(характеристики), допустим 20 строк
top_properties = properties[properties['itemid'].isin(list(events_deal.index))].drop_duplicates(['itemid', 'property']).groupby("property")['itemid'].count().sort_values(ascending=False)[:40]
# Удалим характеристики "категория товара" и "доступность". Без подробного описания, они очевидно неинформативны.
top_properties.drop(['categoryid','available'],inplace=True)
properties['best_feat'] = np.where(properties['property'].isin(list(top_properties.index)),properties['property'],0)
tags = list(set(list(top_properties.index)))
print(f'20 самых частых характеристик среди проданных items: {tags}')

20 самых частых характеристик среди проданных items: ['230', '917', '776', '364', '678', '935', '764', '227', '761', '698', '558', '839', '591', '546', '713', '283', '550', '686', '400', '941', '28', '112', '888', '846', '566', '159', '720', '1066', '348', '790', '19', '810', '1036', '928', '521', '689', '6', '202']


In [67]:
# добавим признак best_feat с наилучшими характеристиками items
properties.loc[:,'best_feat'] = properties['best_feat'].astype('str')
properties.head(3)

,timestamp,itemid,property,value,best_feat
0,1435460400000,460429,categoryid,1338,0
1,1441508400000,206783,888,1116713 960601 n277200,888
2,1439089200000,395014,400,n552000 639502 n720000 424566,400


Преобразуем справочник items. Опишем каждый itemid через список наиболее привлекательных характеристик.

In [68]:
prop_sell = properties.groupby(['itemid'])['best_feat'].agg(' '.join).reset_index()
prop_sell.loc[:,'best_feat_list'] = prop_sell['best_feat'].apply(lambda x:list(set(x.split())))
prop_sell.drop(['best_feat'], axis=1, inplace=True)
prop_sell.head(3)

,itemid,best_feat_list
0,0,"[0, 917, 698, 839, 112, 888, 776, 159, 364, 28..."
1,1,"[0, 698, 917, 839, 112, 888, 776, 159, 364, 67..."
2,2,"[0, 698, 917, 839, 112, 888, 776, 159, 364, 28..."


In [69]:
prop_max = prop_sell[prop_sell['best_feat_list'].apply(lambda x: len(x))==19].shape[0]
print(f'Количество item с полным набором наилучших характеристик: {prop_max}')
prop_ele = prop_sell[prop_sell['best_feat_list'].apply(lambda x: len(x))==11].shape[0]
print(f'Количество item содержащих 11 наилучших характеристик: {prop_ele}')
prop_min = prop_sell[prop_sell['best_feat_list'].apply(lambda x: len(x))==0].shape[0]
print(f'Количество item не содержащих ни одной наилучшей характеристики: {prop_min}')

Количество item с полным набором наилучших характеристик: 38567
Количество item содержащих 11 наилучших характеристик: 9
Количество item не содержащих ни одной наилучшей характеристики: 0


Каждый из items в справочнике содержит не менее 11 лучших характеристик.

In [70]:
# Создадим карту признаков,
for x in tags:
    prop_sell[x] =prop_sell['best_feat_list'].apply(lambda y: str(x) in y)
items_map = prop_sell.drop(['best_feat_list'], axis=1)
items_map.head(3)

,itemid,230,917,776,364,678,935,764,227,761,...,348,790,19,810,1036,928,521,689,6,202
0,0,False,True,True,True,True,False,True,True,False,...,False,True,False,False,True,False,False,False,True,True
1,1,False,True,True,True,True,False,True,True,False,...,False,True,False,False,True,False,False,True,True,True
2,2,False,True,True,True,True,False,True,False,False,...,False,True,False,False,False,False,False,False,False,True


In [71]:
# Заменим значения True/False на названия столбцов
for x in items_map.drop(['itemid'], axis=1).columns:
     items_map[x] = np.where( items_map[x]==True, x,"")

# Получим уникальный список метаданных items
item_metadata = items_map.drop(['itemid'], axis=1).columns
item_metadata

Index(['230', '917', '776', '364', '678', '935', '764', '227', '761', '698',
       '558', '839', '591', '546', '713', '283', '550', '686', '400', '941',
       '28', '112', '888', '846', '566', '159', '720', '1066', '348', '790',
       '19', '810', '1036', '928', '521', '689', '6', '202'],
      dtype='object')

Теперь, когда мы создали наши items features, мы можем переделать все наши mappings.

In [72]:
# events.head(3)

In [73]:
# Создадим mappings для user, item and metadata
dataset = Dataset()
dataset.fit(train_users,
            train_items,
            #на этот раз мы передаем список items
            item_features = item_metadata)

# Сохраним mappings между пользователями и их идентификаторами
# карта идентификаторов пользователей,
user_mappings = dataset.mapping()[0]
# карта характеристик пользователей,
user_metadata_mappings = dataset.mapping()[1]
# карта идентификаторов предметов,
user_metadata_mappings = dataset.mapping()[1]
# карта характеристик предметов
item_mappings = dataset.mapping()[2]
# теперь это отличается от items mapping
item_metadata_mappings = dataset.mapping()[3]
# 1 на каждый item + 1 на metadata
len(item_mappings),len(item_metadata_mappings)

(14422, 14460)

In [74]:
dict(list(item_mappings.items())[:10]) == dict(list(item_metadata_mappings.items())[:10])

True

Последние элементы 2 словарей mappings

In [75]:
dict(list(item_mappings.items())[-10:])

{250640: 14412,
 261886: 14413,
 20222: 14414,
 75035: 14415,
 384734: 14416,
 17127: 14417,
 218917: 14418,
 54141: 14419,
 290723: 14420,
 193218: 14421}

Номера items

In [76]:
dict(list(item_metadata_mappings.items())[-10:])

{'348': 14450,
 '790': 14451,
 '19': 14452,
 '810': 14453,
 '1036': 14454,
 '928': 14455,
 '521': 14456,
 '689': 14457,
 '6': 14458,
 '202': 14459}

Добавились номера feature items

In [77]:
# Создадим обратные mappings
inv_user_mappings = {v:k for k, v in user_mappings.items()}
inv_item_mappings = {v:k for k, v in item_mappings.items()}
inv_item_metadata_mappings = {v:k for k, v in item_metadata_mappings.items()}

# Создадим interactions matrix for each user, item and the weight
train_interactions, train_weights = dataset.build_interactions(train[['userid', 'itemid', 'weights']].values)
train_interactions, train_weights

# Удалим всех new users in the test set
test_interactions, test_weights = dataset.build_interactions(test[['userid', 'itemid']].values)
test_interactions, test_weights

# Создадим test set состоящий только из новых купленых продуктов
test_new_interactions, test_new_weights = dataset.build_interactions(test_new[['userid', 'itemid']].values)
test_new_interactions, test_new_weights

(<16541x14422 sparse matrix of type '<class 'numpy.int32'>'
 	with 585 stored elements in COOrdinate format>,
 <16541x14422 sparse matrix of type '<class 'numpy.float32'>'
 	with 585 stored elements in COOrdinate format>)

Перед передачей данных в модель необходимо оставить только те данные, о которых есть упоминание в train

In [78]:
_ = len(set(items_map['itemid']).difference(set(train_items)))
print(f'Полный справочник items отличается от items входящих в train на {_} элементов')
items_map = items_map[items_map['itemid'].isin(train_items)]
_ = len(set(items_map['itemid']).difference(set(train_items)))
print(f'Полный справочник items отличается от items входящих в train на {_} элементов')

Полный справочник items отличается от items входящих в train на 403040 элементов
Полный справочник items отличается от items входящих в train на 0 элементов


In [79]:
# Создадим поиск items по их характеристикам
item_to_metadata_lookup = [(x[0], list(filter(None, x[1:]))) for x in items_map.values]

# Посмотрим на сопоставление items с tags
item_to_metadata_lookup[:2]

[(15,
  ['917',
   '776',
   '364',
   '678',
   '764',
   '227',
   '698',
   '839',
   '591',
   '283',
   '112',
   '888',
   '159',
   '790',
   '202']),
 (19,
  ['917',
   '776',
   '364',
   '678',
   '764',
   '227',
   '698',
   '839',
   '283',
   '112',
   '888',
   '159',
   '790',
   '689',
   '6',
   '202'])]

In [80]:
# Создание item feature + списка тегов с помощью LightFM
item_metadata_list = dataset.build_item_features(item_to_metadata_lookup, normalize=True)

In [81]:
def objective(trial):

    train, val = random_train_test_split(train_interactions, test_percentage=0.25, random_state=42)
    train_weights, val_weight = random_train_test_split(train_interactions, test_percentage=0.25, random_state=42)

    param = {
        'no_components': trial.suggest_int("no_components", 16, 64),
        "learning_schedule": trial.suggest_categorical("learning_schedule", ["adagrad", "adadelta"]),
        "loss":  trial.suggest_categorical("loss", ["warp"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.5),
        "item_alpha": trial.suggest_float("item_alpha", 1e-10, 1e-06, log=True),
        "user_alpha": trial.suggest_float("user_alpha", 1e-10, 1e-06, log=True),
        "max_sampled": trial.suggest_int("max_sampled", 5, 15),
    }
    epochs = trial.suggest_int("epochs", 20, 50)
    sample_weights = trial.suggest_categorical("sample_weight", ["None", "train_weights"]) # добавим веса как параметр

    model = LightFM(**param, random_state=123)
    model.fit(train,
              sample_weight= eval(sample_weights),
              item_features = item_metadata_list, # используем характеристики items
              epochs = epochs,
              verbose=True)

    val_precision = precision_at_k(model,
                                   val,
                                   train_interactions=train,
                                   item_features = item_metadata_list,
                                   k=3).mean()

    return val_precision

study = optuna.create_study(direction="maximize")

# Add our last run
study.enqueue_trial(best_params)

study.optimize(objective, n_trials=N_TRIAL)
best_params = study.best_params
for k, v in best_params.items():
    print(k,":",v)

[I 2024-06-29 07:58:37,658] A new study created in memory with name: no-name-53f95f9e-dbca-4706-9ca4-b990346a264a
/opt/conda/lib/python3.10/site-packages/optuna/trial/_trial.py:650: UserWarning:

Fixed parameter 'learning_rate' with value 0.826297867169715 is out of range for distribution FloatDistribution(high=0.5, log=False, low=0.001, step=None).

Epoch: 100%|██████████| 31/31 [00:18<00:00,  1.68it/s]
[I 2024-06-29 08:00:02,811] Trial 0 finished with value: 0.0018993354169651866 and parameters: {'no_components': 53, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.826297867169715, 'item_alpha': 2.7834720549771643e-07, 'user_alpha': 2.8829209631989213e-07, 'max_sampled': 7, 'epochs': 31, 'sample_weight': 'train_weights'}. Best is trial 0 with value: 0.0018993354169651866.
Epoch: 100%|██████████| 34/34 [00:09<00:00,  3.47it/s]
[I 2024-06-29 08:01:11,696] Trial 1 finished with value: 0.0014245015336200595 and parameters: {'no_components': 40, 'learning_schedule': 'ad

no_components : 59
learning_schedule : adadelta
loss : warp
learning_rate : 0.007689964487859082
item_alpha : 3.135733006704355e-08
user_alpha : 7.396793165816634e-10
max_sampled : 9
epochs : 38
sample_weight : None


In [82]:
num_epochs = best_params['epochs']
sample_weights = best_params['sample_weight']
del best_params['epochs']
del best_params['sample_weight']
model = LightFM(**best_params, random_state=123)

model.fit(train_interactions,
          sample_weight= eval(sample_weights),
          item_features = item_metadata_list,
          epochs = num_epochs,
          verbose=True)

# Рассчитаем метрики
for metric in [precision_at_k]:
    # Precision@3 для Train and Test
    for data, name in [(train_interactions, "Train"), (test_interactions, "Test ")]:
        print(f"{name} {metric.__name__}: %.2f" %
              metric(model,
                     data,
                     item_features = item_metadata_list,
                     k=3).mean())

    # Покупки только новых items
    print(f"Test new {metric.__name__}: %.3f" %
          metric(model,
                 test_new_interactions,
                 item_features = item_metadata_list,
                 train_interactions=train_interactions, #запретим рекомендовать ранее купленные items
                 k=3).mean())

Epoch: 100%|██████████| 38/38 [00:33<00:00,  1.12it/s]


Train precision_at_k: 0.05
Test  precision_at_k: 0.03
Test new precision_at_k: 0.006


Хуже чем без признаков items

### tems feature 2 version

Следующий вариант - соединить свойства и его описание.  
Опиратся просто на свойство я считаю неправильным.   
Обязательно должно быть значение свойства.   
К примеру, молоко. Его свойство - жирность.   
Но значение имеет какая жирность.   
Соединяя зашифрованнные свойства и их описания мы можем прийти к значению - "Жирность 3.2%" или "Цвет-черный".  
А это и есть искомые потребительские свойства.

In [83]:
#приведем данные о свойствах к текстовому виду
properties['value'] = properties['value'].astype("str")
properties['property'] = properties['property'].astype("str")
#Сгруппируем items по itemsid и property. Агрегируем описания свойств
df1 = properties.groupby(['itemid', 'property'], as_index = False)['value'].agg(' '.join)
# Отсортируем, оставим только купленные items
df1 = df1[df1['itemid'].isin(list(events_deal.index))]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 339680 entries, 398 to 12003662
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   itemid    339680 non-null  int64 
 1   property  339680 non-null  object
 2   value     339680 non-null  object
dtypes: int64(1), object(2)
memory usage: 10.4+ MB


In [84]:
df1.head()

,itemid,property,value
398,15,112,679677
399,15,159,519769
400,15,202,789221
401,15,227,433564
402,15,283,433564 245772 789221 809278 245772 1213953 429...


In [85]:
# Преобразуем данные о свойствах items в удобный для обработки вид.
# Для этого соеденим свойство и его описание для каждого items.
def prop_feature_best(df):
    df_temp =  df.loc[:, ['property','value']].copy()
    len_pf =  df.shape[0]
#     for i in tqdm(range(len_pf)):
    for i in tqdm(list(df.index)):
        list_prop_feat = []
#         print(i)
        split_feat = df_temp.loc[i,'value'].split()
        for f in split_feat:
            list_prop_feat.append(df.loc[i,'property']+'_'+f)
        df_temp.loc[i,'value'] = list_prop_feat
    return df_temp
df_all = prop_feature_best(df1)
df_all

100%|██████████| 339680/339680 [01:31<00:00, 3698.69it/s]


,property,value
398,112,[112_679677]
399,159,[159_519769]
400,202,[202_789221]
401,227,[227_433564]
402,283,"[283_433564, 283_245772, 283_789221, 283_80927..."
...,...,...
12003658,839,[839_705787]
12003659,888,[888_1076947]
12003660,917,"[917_831856, 917_1029906]"
12003661,available,"[available_0, available_1, available_1, availa..."


Найдем 20 наиболее популярных признаков среди фактически купленных items.

In [86]:
df_all.loc[:,'itemid'] = df1.loc[:,'itemid']
list_count= []
list_count = list(df_all['value'].explode())
print(f'Общее количество характеристик: {len(set(list_count))}')


Общее количество характеристик: 124604


Список 20 самых популярных характеристик у items

In [87]:
cnt = Counter(list_count)
dict_feat = dict(cnt)
most_popular_tags = cnt.most_common(40)
# dict_feat.most_common(20)
most_popular_tags[:10]

[('available_1', 90104),
 ('available_0', 54516),
 ('888_1284577', 32560),
 ('888_1297729', 16363),
 ('888_350726', 13579),
 ('400_424566', 13554),
 ('888_30603', 13527),
 ('283_30603', 13270),
 ('888_1154859', 13131),
 ('888_832471', 12244)]

In [88]:
tags = [most_popular_tags[x][0] for x in range(len(most_popular_tags))]
tags[:10]

['available_1',
 'available_0',
 '888_1284577',
 '888_1297729',
 '888_350726',
 '400_424566',
 '888_30603',
 '283_30603',
 '888_1154859',
 '888_832471']

Построим матрицу свойств всего справочника items

In [89]:
df_all.head()

,property,value,itemid
398,112,[112_679677],15
399,159,[159_519769],15
400,202,[202_789221],15
401,227,[227_433564],15
402,283,"[283_433564, 283_245772, 283_789221, 283_80927...",15


In [90]:
df_all = df_all.groupby(['itemid']).agg('sum')
for x in tags:
    df_all[x] =df_all['value'].apply(lambda y: str(x) in y)
items_map = df_all.drop(['value','property'], axis=1)
items_map.head(3)

,available_1,available_0,888_1284577,888_1297729,888_350726,400_424566,888_30603,283_30603,888_1154859,888_832471,...,283_639502,888_150169,888_1135780,28_150169,888_628176,962_664227,689_150169,888_n96000,225_1066831,283_312815
itemid,,,,,,,,,,,,,,,,,,,,,
15,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
25,True,True,False,False,False,False,False,True,True,False,...,False,False,False,True,False,False,False,False,False,True


Теперь у нас есть список характеристик элементов, давайте посмотрим, какие из них являются наиболее распространенными.

In [91]:
# Подсчитаем, как часто встречается каждый тег
item_freqs = df_all.drop(['value','property'], axis=1).sum().sort_values(ascending=False)[:10]
item_freqs

764_1285872    11645
112_679677     11645
159_519769     11645
available_1    11484
available_0     8165
283_30603       6261
283_150169      5520
283_1128577     4036
28_150169       3855
283_237874      3476
dtype: int64

Итак, похоже, что большинство наших items - 112_679677 или 159_519769, а также 764_1285872.  
Большинство из них основное время доступно(available_1)  
Также хорошо представлен признак 283 c описаниями.

In [92]:
# Заменим значения True/False на названия столбцов
for x in df_all.drop(['value','property'], axis=1).columns:
    df_all[x] = np.where(df_all[x]==True, x,"")

# Получение уникального списка метаданных items
item_metadata = df_all.drop(['value','property'], axis=1).columns
item_metadata

Index(['available_1', 'available_0', '888_1284577', '888_1297729',
       '888_350726', '400_424566', '888_30603', '283_30603', '888_1154859',
       '888_832471', '112_679677', '159_519769', '764_1285872', '283_150169',
       '400_n720000', '888_n36000', '888_726612', '888_784581', '283_1128577',
       '888_1187104', '888_86628', '400_639502', '888_n48000', '888_424566',
       '283_237874', '400_n552000', '888_992862', '888_1175087', '888_1318567',
       '888_n12000', '283_639502', '888_150169', '888_1135780', '28_150169',
       '888_628176', '962_664227', '689_150169', '888_n96000', '225_1066831',
       '283_312815'],
      dtype='object')

Теперь, когда мы создали наши items features, мы можем переделать все наши mappings.

In [93]:
# Создадим mappings для user, item and metadata
dataset = Dataset()
dataset.fit(train_users,
            train_items,
            item_features = item_metadata) # this time we pass a lisr of item features to create the index mappings for
# len(dataset.mapping()) #на этот раз мы передаем список элементов, чтобы создать индексные отображения для

# Save the mappings between users and their dummy IDs
# (user id map, user feature map, item id map, item feature map)
# Сохраните сопоставления между пользователями и их фиктивными идентификаторами
# (карта идентификаторов пользователей, карта характеристик пользователей, карта идентификаторов предметов, карта характеристик предметов)
user_mappings = dataset.mapping()[0]
user_metadata_mappings = dataset.mapping()[1]
item_mappings = dataset.mapping()[2]
item_metadata_mappings = dataset.mapping()[3] # this is now different to the items mapping теперь это отличается от отображения элементов

len(item_mappings),len(item_metadata_mappings) # 1 per each item + 1 per metadata

(14422, 14462)

Из вышесказанного видно, что наше сопоставление элементов (индекс к ID элемента) теперь короче, чем наше сопоставление метаданных элемента, которое теперь имеет индекс для каждого ID элемента + каждый ID характеристики. Давайте создадим наши обратные отображения и построим наши взаимодействия. Для данных о признаках LightFM предпочитает иметь список (id пользователя/элемента, [feature1, feature2]) или (id пользователя/элемента, {feature1: feature1_weight, feature2: feature2_weight}). Поскольку мы пока не используем весовые коэффициенты, мы просто создадим список элементов и характеристик.

По умолчанию LightFM нормализует (следит за тем, чтобы их сумма равнялась 1) все признаки в матрице весов. В целом это целесообразно, поскольку мы суммируем все вкрапления для каждого признака, чтобы создать окончательное представление, и хотим, чтобы все наши окончательные представления были примерно в одной шкале. Например, если предмет с 3 признаками имеет итоговое представление в 3 раза больше, чем предмет с 1 признаком, то это потенциально исказит ситуацию при вычислении точечного произведения, поскольку оно чувствительно к базовому размеру вкраплений, например, предмет с большим количеством признаков может получить повышенный балл просто из-за того, что у него много признаков. Именно этого позволяет избежать матрица весов и нормализация. Для нашего элемента с 3 признаками его окончательное представление будет (1/3* признак1) + (1/3 * признак2) + (1/3* признак3) вместо (признак1 + признак2 + признак3).

In [94]:
# Создадим обратные mappings
inv_user_mappings = {v:k for k, v in user_mappings.items()}
inv_item_mappings = {v:k for k, v in item_mappings.items()}
inv_item_metadata_mappings = {v:k for k, v in item_metadata_mappings.items()}

# Создадим interactions matrix for each user, item and the weight
train_interactions, train_weights = dataset.build_interactions(train[['userid', 'itemid', 'weights']].values)
train_interactions, train_weights

# Удалим всех new users in the test set
test_interactions, test_weights = dataset.build_interactions(test[['userid', 'itemid']].values)
test_interactions, test_weights

# Создадим test set состоящий только из новых купленых продуктов
test_new_interactions, test_new_weights = dataset.build_interactions(test_new[['userid', 'itemid']].values)
test_new_interactions, test_new_weights

(<16541x14422 sparse matrix of type '<class 'numpy.int32'>'
 	with 585 stored elements in COOrdinate format>,
 <16541x14422 sparse matrix of type '<class 'numpy.float32'>'
 	with 585 stored elements in COOrdinate format>)

In [95]:
df_all.reset_index(inplace = True)
_ = len(set(df_all['itemid']).difference(set(train_items)))
print(f'Полный справочник items отличается от items входящих в train на {_} элементов')
df_all = df_all[df_all['itemid'].isin(train_items)]
_ = len(set(df_all['itemid']).difference(set(train_items)))
print(f'Полный справочник items отличается от items входящих в train на {_} элементов')


Полный справочник items отличается от items входящих в train на 0 элементов
Полный справочник items отличается от items входящих в train на 0 элементов


In [96]:
# Создадим поиск items по их характеристикам
item_to_metadata_lookup = [(x[0], list(filter(None, x[1:]))) for x in df_all.drop(columns=["value", 'property']).values]

# Посмотрим на сопоставление items с tags
item_to_metadata_lookup[:2]

[(15,
  ['available_1',
   'available_0',
   '112_679677',
   '159_519769',
   '764_1285872',
   '283_1128577',
   '283_237874']),
 (19,
  ['available_1',
   'available_0',
   '112_679677',
   '159_519769',
   '764_1285872',
   '283_150169',
   '689_150169'])]

In [97]:
# Создание item feature + списка тегов с помощью LightFM
item_metadata_list = dataset.build_item_features(item_to_metadata_lookup, normalize=True)

Теперь, когда мы создали наши item features, мы можем переделать все наши mappings.

In [98]:
def objective(trial):

    train, val = random_train_test_split(train_interactions, test_percentage=0.25, random_state=42)
    train_weights, val_weight = random_train_test_split(train_interactions, test_percentage=0.25, random_state=42)

    param = {
        'no_components': trial.suggest_int("no_components", 5, 64),
        "learning_schedule": trial.suggest_categorical("learning_schedule", ["adagrad", "adadelta"]),
        "loss":  trial.suggest_categorical("loss", ["warp"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 1),
        "item_alpha": trial.suggest_float("item_alpha", 1e-10, 1e-06, log=True),
        "user_alpha": trial.suggest_float("user_alpha", 1e-10, 1e-06, log=True),
        "max_sampled": trial.suggest_int("max_sampled", 5, 15),
    }
    epochs = trial.suggest_int("epochs", 20, 50)
    sample_weights = trial.suggest_categorical("sample_weight", ["None", "train_weights"]) # добавим веса как параметр

    model = LightFM(**param, random_state=123)
    model.fit(train,
              sample_weight= eval(sample_weights),
              item_features = item_metadata_list, # use our item features
              epochs = epochs,
              verbose=True)

    val_precision = precision_at_k(model,
                                   val,
                                   train_interactions=train,
                                   item_features = item_metadata_list,
                                   k=3).mean()

    return val_precision

study = optuna.create_study(direction="maximize")

# # Add in our original hyperparmeter values as a starting point for Optuna
best_params["epochs"]=20 # manually add epochs
study.enqueue_trial(best_params)

study.optimize(objective, n_trials=N_TRIAL)
best_params = study.best_params
for k, v in best_params.items():
    print(k,":",v)

[I 2024-06-29 08:23:32,089] A new study created in memory with name: no-name-55e62c61-c90c-41c8-9928-00403a70dd62
Epoch: 100%|██████████| 20/20 [00:07<00:00,  2.58it/s]
[I 2024-06-29 08:24:17,038] Trial 0 finished with value: 0.0029846697580069304 and parameters: {'no_components': 59, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.007689964487859082, 'item_alpha': 3.135733006704355e-08, 'user_alpha': 7.396793165816634e-10, 'max_sampled': 9, 'epochs': 20, 'sample_weight': 'None'}. Best is trial 0 with value: 0.0029846697580069304.
Epoch: 100%|██████████| 20/20 [00:02<00:00,  7.40it/s]
[I 2024-06-29 08:24:38,953] Trial 1 finished with value: 6.78334035910666e-05 and parameters: {'no_components': 29, 'learning_schedule': 'adagrad', 'loss': 'warp', 'learning_rate': 0.8776717852309891, 'item_alpha': 2.247842606186568e-08, 'user_alpha': 8.879244137843594e-08, 'max_sampled': 8, 'epochs': 20, 'sample_weight': 'train_weights'}. Best is trial 0 with value: 0.0029846697580069

no_components : 21
learning_schedule : adadelta
loss : warp
learning_rate : 0.08565088055508488
item_alpha : 3.377584843434072e-07
user_alpha : 3.541693311906726e-07
max_sampled : 11
epochs : 43
sample_weight : None


In [99]:
num_epochs = best_params['epochs']
sample_weights = best_params['sample_weight']
del best_params['epochs']
del best_params['sample_weight']

# Train with the best parameters
model = LightFM(**best_params, random_state=123)

model.fit(train_interactions,
          sample_weight= eval(sample_weights),
          item_features = item_metadata_list,
          epochs = num_epochs,
          verbose=True)

# Measure how well it did in the Test period
for metric in [precision_at_k]:
    # Get the precision and recall for Train and Test
    for data, name in [(train_interactions, "Train"), (test_interactions, "Test ")]:
        print(f"{name} {metric.__name__}: %.2f" %
              metric(model,
                     data,
                     item_features = item_metadata_list,
                     k=3).mean())

    # What about for just new-to-user purchases?
    print(f"Test new {metric.__name__}: %.3f" %
          metric(model,
                 test_new_interactions,
                 item_features = item_metadata_list,
                 train_interactions=train_interactions, # supress previously bought prods from being recommended
                 k=3).mean())

Epoch: 100%|██████████| 43/43 [00:08<00:00,  4.87it/s]


Train precision_at_k: 0.12
Test  precision_at_k: 0.06
Test new precision_at_k: 0.007


Добавление характеристик items не улучшило предсказательную способность модели, попробуем другой подход к feature engineering

### item feature 3 ver

Особенность данного подхода в том, что мы изменим вес признаков items.     
На данный момент вес элемента и его признаков одинаков, поэтому если у элемента 20 признака, то итоговое представление этого элемента будет таким: 1/20 вложения идентичности элемента + 19/20 вложения признаков.   
Один из способов изменить весовые коэффициенты в пользу более выразительной модели, сохранив при этом возможность делать предсказания с холодного старта и разумные предложения по элементам, - это уменьшить вес признаков в итоговом представлении,
Возможно за счет этого качество предсказания при использовании характеристик элементов значительно снижается.
Я хочу повысить вес основного признака и снизить вес его характеристик.  
Для этого нам сначала нужно создать фрейм данных, в котором для каждого товара будут указаны все уникальные характеристики, связанные с ним в виде длинной текстовой строки.

In [100]:
# Try tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

text_data = pd.DataFrame(zip([(i[0]) for i in item_to_metadata_lookup],
                            		 		[' '.join(i[1]) for i in item_to_metadata_lookup]),
                         					columns=(["product_name", "description"]))
list(text_data[text_data['product_name'] == 147]['description'])

['available_1 888_1284577 283_30603 112_679677 159_519769 764_1285872 283_150169 888_n36000 283_1128577 283_237874 888_n12000']

In [101]:
text_data.loc[:,'description'] = text_data['description'].str.replace('.', '', regex = False)

In [102]:
list(text_data[text_data['product_name'] == 147]['description'])

['available_1 888_1284577 283_30603 112_679677 159_519769 764_1285872 283_150169 888_n36000 283_1128577 283_237874 888_n12000']

Они выглядят довольно хорошо.  
еперь мы обработали наши текстовые данные и можем вызвать TfidfVectorizer(), чтобы создать весовые коэффициенты для каждого из тегов для каждого из товаров.  
Приведенный ниже код создает рамку данных pandas со строкой для каждого товара и столбцом для каждой характеристики товара, например "сверкающий".   
Значение столбца - это соответствующий tf-idf вес для данного товара и характеристики товара.   
Затем мы можем просмотреть каждую строку и вернуть словарь с характеристикой товара и его весом, отфильтровав те случаи, когда вес >0:

In [103]:
tfidfvectorizer = TfidfVectorizer()
tfidf_weights = tfidfvectorizer.fit_transform(text_data["description"])
tfidf_tokens = tfidfvectorizer.get_feature_names_out()
tfidf = pd.DataFrame(data = tfidf_weights.toarray(), index = text_data['product_name'], columns = tfidf_tokens).reset_index()

In [104]:
tfidf_weights =[]
for x in tfidf.values:
    tfidf_weights.append(tuple([x[0], dict(list(filter(lambda item: item[1] > 0, zip(tfidf.columns[1:], x[1:]))))]))

In [105]:
tfidf_weights[:3]

[(15.0,
  {'112_679677': 0.2583353124824802,
   '159_519769': 0.2583353124824802,
   '283_1128577': 0.5320314832959111,
   '283_237874': 0.570609209297835,
   '764_1285872': 0.2583353124824802,
   'available_0': 0.3500401066816297,
   'available_1': 0.2619315821631736}),
 (19.0,
  {'112_679677': 0.26849908424265523,
   '159_519769': 0.26849908424265523,
   '283_150169': 0.4689078129932667,
   '689_150169': 0.5978111966488328,
   '764_1285872': 0.26849908424265523,
   'available_0': 0.36381185053279474,
   'available_1': 0.2722368431524879}),
 (25.0,
  {'112_679677': 0.17071361776188412,
   '159_519769': 0.17071361776188412,
   '283_1128577': 0.35157802626315743,
   '283_30603': 0.27663512687350456,
   '283_312815': 0.4731090634070069,
   '28_150169': 0.3594089100262583,
   '764_1285872': 0.17071361776188412,
   '888_1154859': 0.5253219084448664,
   'available_0': 0.2313141490381012,
   'available_1': 0.17309011132654217})]

In [106]:
tfidf_item_list = dataset.build_item_features(tfidf_weights, normalize=True)
tfidf_weights[:3]

[(15.0,
  {'112_679677': 0.2583353124824802,
   '159_519769': 0.2583353124824802,
   '283_1128577': 0.5320314832959111,
   '283_237874': 0.570609209297835,
   '764_1285872': 0.2583353124824802,
   'available_0': 0.3500401066816297,
   'available_1': 0.2619315821631736}),
 (19.0,
  {'112_679677': 0.26849908424265523,
   '159_519769': 0.26849908424265523,
   '283_150169': 0.4689078129932667,
   '689_150169': 0.5978111966488328,
   '764_1285872': 0.26849908424265523,
   'available_0': 0.36381185053279474,
   'available_1': 0.2722368431524879}),
 (25.0,
  {'112_679677': 0.17071361776188412,
   '159_519769': 0.17071361776188412,
   '283_1128577': 0.35157802626315743,
   '283_30603': 0.27663512687350456,
   '283_312815': 0.4731090634070069,
   '28_150169': 0.3594089100262583,
   '764_1285872': 0.17071361776188412,
   '888_1154859': 0.5253219084448664,
   'available_0': 0.2313141490381012,
   'available_1': 0.17309011132654217})]

Теперь мы видим, что у нашего первого товара "Mirabella Rose Brut" есть 3 характеристики: "Rose", "Brut" и "Sparkling". Мы видим, что "Розовое" получило наивысшую оценку, что вполне логично. Тот факт, что это игристое вино, безусловно, важен, но то, что это еще и роза, вероятно, важнее. Теперь попробуем запустить optuna с нашей матрицей элементов tf-idf в качестве одного из возможных гиперпараметров:

In [107]:
def objective(trial):

    train, val = random_train_test_split(train_interactions, test_percentage=0.25, random_state=42)

    param = {
        'no_components': trial.suggest_int("no_components", 5, 64),
        "learning_schedule": trial.suggest_categorical("learning_schedule", ["adagrad", "adadelta"]),
        "loss":  trial.suggest_categorical("loss", ["warp"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 1),
        "item_alpha": trial.suggest_float("item_alpha", 1e-10, 1e-06, log=True),
        "user_alpha": trial.suggest_float("user_alpha", 1e-10, 1e-06, log=True),
        "max_sampled": trial.suggest_int("max_sampled", 5, 15),
    }
    epochs = trial.suggest_int("epochs", 20, 50)
    item_features = trial.suggest_categorical("item_weights", ["item_metadata_list", "tfidf_item_list"])

    model = LightFM(**param, random_state=123)
    model.fit(train,
              item_features = eval(item_features),
              epochs = epochs,
              verbose=True)

    val_precision = precision_at_k(model,
                                   val,
                                   train_interactions=train,
                                   item_features = eval(item_features),
                                   k=10).mean()

    return val_precision

study = optuna.create_study(direction="maximize")

# Add our last run
best_params["epochs"]=num_epochs # manually add epochs
best_params["item_weights"]="item_metadata_list" # and feature weights
study.enqueue_trial(best_params)

# Run the study
study.optimize(objective, n_trials=N_TRIAL)

best_params = study.best_params
for k, v in best_params.items():
    print(k,":",v)

[I 2024-06-29 08:30:38,023] A new study created in memory with name: no-name-6148c7db-8fc0-4cff-913b-dac17d4b45d3
Epoch: 100%|██████████| 43/43 [00:06<00:00,  6.76it/s]
[I 2024-06-29 08:30:58,563] Trial 0 finished with value: 0.0018315018387511373 and parameters: {'no_components': 21, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.08565088055508488, 'item_alpha': 3.377584843434072e-07, 'user_alpha': 3.541693311906726e-07, 'max_sampled': 11, 'epochs': 43, 'item_weights': 'item_metadata_list'}. Best is trial 0 with value: 0.0018315018387511373.
Epoch: 100%|██████████| 46/46 [00:10<00:00,  4.35it/s]
[I 2024-06-29 08:31:41,134] Trial 1 finished with value: 0.0013431013794615865 and parameters: {'no_components': 44, 'learning_schedule': 'adagrad', 'loss': 'warp', 'learning_rate': 0.5514243759039766, 'item_alpha': 2.3933592314695223e-07, 'user_alpha': 3.028140715142627e-10, 'max_sampled': 12, 'epochs': 46, 'item_weights': 'item_metadata_list'}. Best is trial 0 with value

no_components : 22
learning_schedule : adagrad
loss : warp
learning_rate : 0.044214405561134394
item_alpha : 2.5229325057713554e-09
user_alpha : 3.285855361326423e-09
max_sampled : 13
epochs : 46
item_weights : item_metadata_list


В этом случае оригинальные равномерно разделенные веса работают лучше (по крайней мере, при том количестве испытаний, которое мы провели). Это позволяет сохранить простоту и легкость объяснения итоговой модели, что несомненный плюс! Давайте обучим лучшую модель из наших испытаний и посмотрим, как она работает в целом.

In [108]:
num_epochs = best_params['epochs']
item_features = best_params['item_weights']
del best_params['epochs']
del best_params['item_weights']

# Which parameters were the most important?
optuna.importance.get_param_importances(study)
fig = optuna.visualization.plot_param_importances(study)
fig.show()

# Train with the best parameters
model = LightFM(**best_params, random_state=123)

model.fit(train_interactions,
          item_features = eval(item_features),
          epochs = num_epochs,
          verbose=True)

# Measure how well it did in the Test period
for metric in [precision_at_k]:
    # Get the precision and recall for Train and Test
    for data, name in [(train_interactions, "Train"), (test_interactions, "Test ")]:
        print(f"{name} {metric.__name__}: %.2f" %
              metric(model,
                     data,
                     item_features =  eval(item_features),
                     k=3).mean())
           # Покупки только новых items
        print(f"Test new {metric.__name__}: %.3f" %
              metric(model,
                     test_new_interactions,
                     item_features = item_metadata_list,
                     train_interactions=train_interactions, #запретим рекомендовать ранее купленные items
                     k=3).mean())


Epoch: 100%|██████████| 46/46 [00:09<00:00,  4.78it/s]


Train precision_at_k: 0.07
Test new precision_at_k: 0.002
Test  precision_at_k: 0.03
Test new precision_at_k: 0.002


# Рекомендации для items с холодным стартом

Эмбеддинги для характеристик items позволяют давать рекомендации для новых или "холодных" items.    
Эти предметы не имеют никаких взаимодействий с пользователем, поэтому мы не можем создавать эмбеддинги для них напрямую. Однако мы можем выразить item в терминах характеристик, для которых у нас есть эмбеддинги.    
Сначала давайте получим индексы признаков элементов для каждого из наших атрибутов.
В качестве примера снова возьмем

In [109]:
random.seed(42)
# Получим indexes for the feature combinations we want to return embeddings for
new_item_attriutes = random.sample(list(item_metadata),k=5)
print(new_item_attriutes)
new_item_indexes = [item_metadata_mappings.get(key) for key in new_item_attriutes]
new_item_indexes

['283_30603', 'available_0', '888_784581', '888_n36000', '400_n720000']


[14429, 14423, 14439, 14437, 14436]

Мы можем создать веса для каждого из индекса.
Для простоты мы просто присвоим им всем одинаковый вес, который будет равен 1/количеству характеристик.
Следующей частью является создание строки поиска для нашего элемента, которая имитирует обычную матрицу характеристик элемента, которую привык получать LightFM.  
Мы создаем массив из всех 0, который соответствует длине уже существующих характеристик элемента.  
Затем мы перезаписываем в каждом индексе нашей характеристики 0 весом нашей характеристики. В качестве проверки мы можем просуммировать строки, чтобы убедиться, что наши веса равны 1.

In [110]:
# Can just weight each attribute equally
weights = 1/len(new_item_indexes) # weight each metadata equally
std_weights = [[weights] * len(new_item_attriutes)]

new_item = np.zeros(len(item_metadata_mappings)) # create an empty array that will serve as our dummy cold-user row
np.put(new_item, new_item_indexes, std_weights) # update the relevant metadata attributes with the desired weights
new_item.sum()

1.0

Теперь, когда мы создали нашу строку признаков item с холодным стартом, мы можем преобразовать ее в разреженную матрицу и передать ее в LightFM.   
Мы можем использовать функцию get_representations() из LightFM, чтобы вычислить сумму(веса*семена) для нашего элемента, а затем вычислить косинусное сходство между ним и другими элементами.

In [111]:
# Convert it into a sparse matrix
cold_item_matrix = scipy.sparse.csr_matrix(new_item)

# Use LightFM to convert the matrix into embeddings
cold_item_bias, cold_item_embedding = model.get_item_representations(cold_item_matrix)
item_biases, item_embeddings  = model.get_item_representations(features = item_metadata_list)

# Находим похожие items
item_item_cold = pd.DataFrame(cosine_similarity(cold_item_embedding, item_embeddings).T, columns=(["cosine"]))
item_item_cold["item_name"]=item_item_cold.index.to_series().map(inv_item_mappings)
item_item_cold.sort_values(by="cosine", ascending=False)[:10]

,cosine,item_name
4453,0.817924,360905
3282,0.815696,351435
2998,0.808639,46008
4664,0.807250,720
24,0.805898,402047
12582,0.805076,346204
11920,0.804607,143866
12923,0.803314,465333
13691,0.800540,122934
2645,0.798207,307074


Таким образом, мы можем найти users, которые купили эти items, и порекомендовать им наш новый item на том основании, что они похожи, поэтому он должен понравиться и этим users.

Если мы хотим создавать рекомендации непосредственно для пользователей, мы можем сделать и это.
Мы просто возьмем наши эмбеддинги холодного элемента и вычислим точечное произведение на эмбеддинги пользователей, чтобы создать рекомендательный балл для нашего нового элемента.
Затем мы можем приложить его ко всем ранее сделанным прогнозам и пересмотреть их, чтобы найти пользователей, для которых этот элемент занимает высокое место.
Обратите внимание, что мы вычисляем ранги для каждого пользователя, а не просто берем наивысший балл для холодного элемента, поскольку фактические баллы в LightFM имеют значение только относительно каждого пользователя как средство ранжирования элементов, но не между пользователями.

In [112]:
# Create all user and item matrix to get predictions for it
n_users, n_items = train_interactions.shape

# Force lightFM to create predictions for all users and all items
scoring_user_ids = np.concatenate([np.full((n_items, ), i) for i in range(n_users)]) # repeat user ID for number of prods
scoring_item_ids = np.concatenate([np.arange(n_items) for i in range(n_users)]) # repeat entire range of item IDs x number of user
scores = model.predict(user_ids = scoring_user_ids,
                                     item_ids = scoring_item_ids)
scores = scores.reshape(-1, n_items) # get 1 row per user
recommendations = pd.DataFrame(scores)
recommendations.shape

(16541, 14422)

In [113]:
# Generate recommendations for all users and then append the prediction for our new product and re-rank
recommendations # all recommendations calculated earlier
# Extract the user and item representations
user_biases, user_embeddings  = model.get_user_representations()
# Create prediction score for our 'new' item
recommendations["cold_ranking"] = ((user_embeddings @ cold_item_embedding.T + cold_item_bias).T + user_biases).T
recommendations.rank(axis=1, ascending=False)  # Highest value gets ranked as 1 i.e. best rec
cold_rankings = recommendations.rank(axis=1, ascending=False)[["cold_ranking"]]
cold_rankings

# Add on users
cold_rankings["user_id"]=cold_rankings.index.to_series().map(inv_user_mappings)
cold_rankings.sort_values(by="cold_ranking")[:10]

,cold_ranking,user_id
15381,52.0,1309591
4426,71.0,379562
5576,74.0,477826
14665,74.0,1249422
8018,78.0,686253
2820,86.0,239564
15961,91.0,1357459
6414,93.0,548918
3494,94.0,297639
618,95.0,53982


# LightFM with users feature

Построение модели рекомендаций с использованием характеристик items и users
Подключим к модели свойства userid.   
Явных свойств users у нас нет. Типа возраста или профессии.   
Поэтому, в качестве таковых, предлагаю использовать время, когда user совершал покупку.   
Преобразуем данные о транзакциях с помощью OneHotEncoder

In [114]:
events = pd.read_csv('/kaggle/input/events.csv/events.csv')
events.drop_duplicates(inplace=True)
events = events[events.index.isin(events_c.index)]
events.reset_index(drop=True, inplace=True)
events['event_datetime'] = pd.to_datetime(events['timestamp'], unit='ms', origin='unix')
# считаю целесообразным использовать признаки месяца, дня недели и времени суток покупки
events['day_of_week'] = events['event_datetime'].map(lambda x: x.weekday())
events['Month'] = events['event_datetime'].map(lambda x: x.month)
events['Hour'] = events['event_datetime'].map(lambda x: x.hour)

def get_time_periods(hour):
    if hour >= 3 and hour < 7:
        return 'Dawn'
    elif hour >= 7 and hour < 12:
        return 'Morning'
    elif hour >= 12 and hour < 16:
        return 'Afternoon'
    elif hour >= 16 and hour < 22:
        return 'Evening'
    else:
        return 'Night'

events['Day Period'] = events['Hour'].map(get_time_periods)
events['Day Period'].value_counts()

Day Period
Evening      17423
Night        12446
Dawn          8117
Afternoon     4878
Morning       2045
Name: count, dtype: int64

In [115]:
events

,timestamp,visitorid,event,itemid,transactionid,event_datetime,day_of_week,Month,Hour,Day Period
0,1433192144515,733007,view,188148,NaN,2015-06-01 20:55:44.515,0,6,20,Evening
1,1433193800251,1054451,view,276105,NaN,2015-06-01 21:23:20.251,0,6,21,Evening
2,1433220834470,994896,view,10572,NaN,2015-06-02 04:53:54.470,1,6,4,Dawn
3,1433217668704,1299250,view,422948,NaN,2015-06-02 04:01:08.704,1,6,4,Dawn
4,1433175360916,234513,view,216695,NaN,2015-06-01 16:16:00.916,0,6,16,Evening
...,...,...,...,...,...,...,...,...,...,...
44904,1438404600106,98537,view,262105,NaN,2015-08-01 04:50:00.106,5,8,4,Dawn
44905,1438401301369,699635,view,207749,NaN,2015-08-01 03:55:01.369,5,8,3,Dawn
44906,1438397263804,532307,view,89232,NaN,2015-08-01 02:47:43.804,5,8,2,Night
44907,1438397312078,1139675,view,432152,NaN,2015-08-01 02:48:32.078,5,8,2,Night


In [116]:
user_biases.shape

(16541,)

In [117]:
one_hot_encoder = OneHotEncoder(dtype=bool)
# one_hot_encoder = OneHotEncoder()
columns_to_change = ['day_of_week','Month','Day Period','event']
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(events[columns_to_change]).toarray()
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
encoder_df = pd.DataFrame(data_onehot, columns = column_names)
#merge one-hot encoded columns back with original DataFrame
final_df = events.join(encoder_df)
final_df

,timestamp,visitorid,event,itemid,transactionid,event_datetime,day_of_week,Month,Hour,Day Period,...,Month_8,Month_9,Day Period_Afternoon,Day Period_Dawn,Day Period_Evening,Day Period_Morning,Day Period_Night,event_addtocart,event_transaction,event_view
0,1433192144515,733007,view,188148,NaN,2015-06-01 20:55:44.515,0,6,20,Evening,...,False,False,False,False,True,False,False,False,False,True
1,1433193800251,1054451,view,276105,NaN,2015-06-01 21:23:20.251,0,6,21,Evening,...,False,False,False,False,True,False,False,False,False,True
2,1433220834470,994896,view,10572,NaN,2015-06-02 04:53:54.470,1,6,4,Dawn,...,False,False,False,True,False,False,False,False,False,True
3,1433217668704,1299250,view,422948,NaN,2015-06-02 04:01:08.704,1,6,4,Dawn,...,False,False,False,True,False,False,False,False,False,True
4,1433175360916,234513,view,216695,NaN,2015-06-01 16:16:00.916,0,6,16,Evening,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44904,1438404600106,98537,view,262105,NaN,2015-08-01 04:50:00.106,5,8,4,Dawn,...,True,False,False,True,False,False,False,False,False,True
44905,1438401301369,699635,view,207749,NaN,2015-08-01 03:55:01.369,5,8,3,Dawn,...,True,False,False,True,False,False,False,False,False,True
44906,1438397263804,532307,view,89232,NaN,2015-08-01 02:47:43.804,5,8,2,Night,...,True,False,False,False,False,False,True,False,False,True
44907,1438397312078,1139675,view,432152,NaN,2015-08-01 02:48:32.078,5,8,2,Night,...,True,False,False,False,False,False,True,False,False,True


In [118]:
final_df = final_df.drop(columns = ['timestamp','event_datetime','itemid', 'transactionid','event','day_of_week','Month','Hour','Day Period']).rename(columns={'visitorid': 'userid'})
# суммируем значения новых признаков users
final_df = final_df.groupby(['userid']).agg('max')
# оставим для модели только тех users, которые присутствуют в train
user_attributes = final_df[final_df.index.isin(train_users)]
# user_attributes.reset_index(inplace=True)
user_metadata = user_attributes.columns
user_metadata

Index(['day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3',
       'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'Month_5', 'Month_6',
       'Month_7', 'Month_8', 'Month_9', 'Day Period_Afternoon',
       'Day Period_Dawn', 'Day Period_Evening', 'Day Period_Morning',
       'Day Period_Night', 'event_addtocart', 'event_transaction',
       'event_view'],
      dtype='object')

In [119]:
user_attributes.reset_index(inplace=True)
user_attributes

,userid,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,Month_5,Month_6,...,Month_8,Month_9,Day Period_Afternoon,Day Period_Dawn,Day Period_Evening,Day Period_Morning,Day Period_Night,event_addtocart,event_transaction,event_view
0,172,False,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
1,627,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True
2,2019,False,False,True,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
3,2242,False,False,False,False,False,False,True,False,True,...,False,False,False,True,False,False,False,False,False,True
4,2646,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,1405714,False,False,False,False,False,True,False,True,False,...,False,False,False,False,True,False,False,True,False,False
2447,1405831,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
2448,1405929,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,True
2449,1406564,False,False,True,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,True


In [120]:
# Create inverse mappings
inv_user_mappings = {v:k for k, v in user_mappings.items()}
inv_user_metadata_mappings = {v:k for k, v in user_metadata_mappings.items()}
inv_item_mappings = {v:k for k, v in item_mappings.items()}
inv_item_metadata_mappings = {v:k for k, v in item_metadata_mappings.items()}

# Заменим значения True/False на названия столбцов
for x in user_attributes.drop(['userid'], axis=1).columns:
    user_attributes[x] = np.where(user_attributes[x]==True, x, "")
# # Заменим значения True/False на названия столбцов
# for x in items_map.drop(['itemid'], axis=1).columns:
#      items_map[x] = np.where( items_map[x]==True, x,"")

/tmp/ipykernel_33/1003904512.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [121]:
# user_attributes.reset_index(inplace=True)

In [122]:
user_attributes

,userid,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,Month_5,Month_6,...,Month_8,Month_9,Day Period_Afternoon,Day Period_Dawn,Day Period_Evening,Day Period_Morning,Day Period_Night,event_addtocart,event_transaction,event_view
0,172,,,day_of_week_2,,,,,,,...,,,,Day Period_Dawn,,,,,,event_view
1,627,,,,day_of_week_3,,,,,,...,Month_8,,,,,,Day Period_Night,,,event_view
2,2019,,,day_of_week_2,,,,,,Month_6,...,,,,,Day Period_Evening,,,,,event_view
3,2242,,,,,,,day_of_week_6,,Month_6,...,,,,Day Period_Dawn,,,,,,event_view
4,2646,,day_of_week_1,,,,,,,,...,,,,,,,Day Period_Night,event_addtocart,,event_view
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,1405714,,,,,,day_of_week_5,,Month_5,,...,,,,,Day Period_Evening,,,event_addtocart,,
2447,1405831,,,,,day_of_week_4,,,,,...,,,,,,,Day Period_Night,,event_transaction,
2448,1405929,,,,day_of_week_3,,,,,,...,,Month_9,,,,,Day Period_Night,,,event_view
2449,1406564,,,day_of_week_2,,,,,,,...,Month_8,,,,Day Period_Evening,,,,,event_view


In [123]:
# Create user, item and metadata mappings
dataset = Dataset()
dataset.fit(train_users,
            train_items,
            user_features = user_metadata,
            item_features = item_metadata)

# Сохраним mappings между пользователями и их идентификаторами
# карта идентификаторов пользователей,
user_mappings = dataset.mapping()[0]
# карта характеристик пользователей,
user_metadata_mappings = dataset.mapping()[1]
# карта идентификаторов предметов
item_mappings = dataset.mapping()[2]
# карта характеристик предметов
# теперь это отличается от items mapping
item_metadata_mappings = dataset.mapping()[3]
# 1 на каждый item + 1 на metadata
len(user_mappings),len(user_metadata_mappings)

(16541, 16561)

In [124]:
# Создадим поиск users по их характеристикам
user_to_metadata_lookup = [(x[0], list(filter(None, x[1:]))) for x in user_attributes.values]

# Посмотрим на сопоставление users с tags
user_to_metadata_lookup[:5]

[(172, ['day_of_week_2', 'Month_7', 'Day Period_Dawn', 'event_view']),
 (627, ['day_of_week_3', 'Month_8', 'Day Period_Night', 'event_view']),
 (2019, ['day_of_week_2', 'Month_6', 'Day Period_Evening', 'event_view']),
 (2242, ['day_of_week_6', 'Month_6', 'Day Period_Dawn', 'event_view']),
 (2646,
  ['day_of_week_1',
   'Month_7',
   'Day Period_Night',
   'event_addtocart',
   'event_view'])]

In [125]:
user_metadata_list = dataset.build_user_features(user_to_metadata_lookup, normalize=True)

In [126]:
_ = len(set(user_attributes['userid']).difference(set(train_users)))
print(f'Полный справочник users отличается от users входящих в train на {_} элементов')

Полный справочник users отличается от users входящих в train на 0 элементов


In [127]:
def objective(trial):

    train, val = random_train_test_split(train_interactions, test_percentage=0.25, random_state=42)
    train_weights, val_weight = random_train_test_split(train_interactions, test_percentage=0.25, random_state=42)

    param = {
        'no_components': trial.suggest_int("no_components", 16, 64),
        "learning_schedule": trial.suggest_categorical("learning_schedule", ["adagrad", "adadelta"]),
        "loss":  trial.suggest_categorical("loss", ["warp"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.5),
        "item_alpha": trial.suggest_float("item_alpha", 1e-10, 1e-06, log=True),
        "user_alpha": trial.suggest_float("user_alpha", 1e-10, 1e-06, log=True),
        "max_sampled": trial.suggest_int("max_sampled", 5, 15),
    }
    epochs = trial.suggest_int("epochs", 20, 50)
    sample_weights = trial.suggest_categorical("sample_weight", ["None", "train_weights"]) # добавим веса как параметр

    model = LightFM(**param, random_state=123)
    model.fit(train,
              sample_weight= eval(sample_weights),
              user_features = user_metadata_list,
              item_features = item_metadata_list,
              epochs = epochs,
              verbose=True)

    val_precision = precision_at_k(model,
                                   val,
                                   train_interactions=train,
                                   user_features = user_metadata_list,
                                   item_features = item_metadata_list,
                                   k=3).mean()

    return val_precision

study = optuna.create_study(direction="maximize")

# Add our last run
study.enqueue_trial(best_params)

# Run the study
study.optimize(objective, n_trials=N_TRIAL)

best_params = study.best_params
# num_epochs = best_params['epochs']
for k, v in best_params.items():
    print(k,":",v)

[I 2024-06-29 08:39:36,265] A new study created in memory with name: no-name-341e651a-9f5c-404b-845a-50c192c98b90
Epoch: 100%|██████████| 26/26 [00:04<00:00,  6.01it/s]
[I 2024-06-29 08:39:56,556] Trial 0 finished with value: 0.004612671677023172 and parameters: {'no_components': 22, 'learning_schedule': 'adagrad', 'loss': 'warp', 'learning_rate': 0.044214405561134394, 'item_alpha': 2.5229325057713554e-09, 'user_alpha': 3.285855361326423e-09, 'max_sampled': 13, 'epochs': 26, 'sample_weight': 'train_weights'}. Best is trial 0 with value: 0.004612671677023172.
Epoch: 100%|██████████| 25/25 [00:03<00:00,  6.31it/s]
[I 2024-06-29 08:40:21,953] Trial 1 finished with value: 0.004341337829828262 and parameters: {'no_components': 30, 'learning_schedule': 'adagrad', 'loss': 'warp', 'learning_rate': 0.1175540606785687, 'item_alpha': 4.735976578855641e-08, 'user_alpha': 1.0941030479167702e-07, 'max_sampled': 5, 'epochs': 25, 'sample_weight': 'train_weights'}. Best is trial 0 with value: 0.0046126

no_components : 16
learning_schedule : adagrad
loss : warp
learning_rate : 0.18447680804536035
item_alpha : 1.2513821225283866e-10
user_alpha : 3.1290355502818705e-09
max_sampled : 10
epochs : 33
sample_weight : None


In [128]:
# сохраним num_epochs как отдельный объект
num_epochs = best_params['epochs']
sample_weights = best_params['sample_weight']
del best_params['epochs']
del best_params['sample_weight']

# Построим модель с наилучшими параметрами
model = LightFM(**best_params, random_state=123)

model.fit(train_interactions,
          sample_weight= eval(sample_weights),
          user_features = user_metadata_list,
          item_features = item_metadata_list,
          epochs = num_epochs,
          verbose=True)

# Рассчитаем метрики
for metric in [precision_at_k]:
    # Precision@3 для Train and Test
    for data, name in [(train_interactions, "Train"), (test_interactions, "Test ")]:
        print(f"{name} {metric.__name__}: %.2f" %
              metric(model,
                     data,
                     user_features = user_metadata_list,
                     item_features = item_metadata_list,
                     k=3).mean())

    # Покупки только новых items
    print(f"Test new {metric.__name__}: %.3f" %
          metric(model,
                 test_new_interactions,
                 user_features = user_metadata_list,
                 item_features = item_metadata_list,
                 train_interactions=train_interactions, #запретим рекомендовать ранее купленные items
                 k=3).mean())

Epoch: 100%|██████████| 33/33 [00:04<00:00,  7.76it/s]


Train precision_at_k: 0.09
Test  precision_at_k: 0.04
Test new precision_at_k: 0.001


# Рекомендации для пользователей холодного старта

Теперь, когда мы добавили пользовательские характеристики, мы можем делать прогнозы для пользователей холодного старта точно так же, как мы делали это для товаров холодного старта.   
Мы просто создаем пользовательскую матрицу атрибутов пользователя, в которой все характеристики личности пользователя равны 0, но мы заполняем ее соответствующими весами для категорий, в которых наш новый пользователь совершал покупки ранее.   
Затем мы можем использовать LightFM для создания представления пользователя, которое мы можем передать для прогнозирования.

In [129]:
random.seed(42)
# Получим indexes for the feature combinations we want to return embeddings for
new_user_attriutes = random.sample(list(user_metadata),k=5)
print(new_user_attriutes)
user_indexes = [item_metadata_mappings.get(key) for key in new_item_attriutes]
user_indexes

['day_of_week_3', 'day_of_week_0', 'Month_6', 'Month_5', 'Day Period_Night']


[14429, 14423, 14439, 14437, 14436]

In [130]:
# # First build a bespoke user_feature_matrix
# for new_user_attriutes in [['beers coolers', 'spirits'], ['asian foods'], ['specialty cheeses','tofu meat alternatives', 'seafood counter','nuts seeds dried fruit']]:
#     user_indexes = [user_metadata_mappings.get(key) for key in new_user_attriutes]

    # Can either just weight each attribute equally
weights = 1/len(user_indexes) # weight each metadata equally
std_weights = [[weights] * len(new_user_attriutes)]

    # Or can pull it from the inv weights
    # weights = [inv_user_weights.get(key) for key in new_user_attriutes]
    # std_weights = [x/(sum(weights)/1) for x in weights]

    # Combine the indexes we want populating with their weights
new_user = np.zeros(len(user_metadata_mappings)) # create an empty array that will server as our dummy cold-user row
np.put(new_user, user_indexes, std_weights) # update the relevant metadata attributes with the desired weights
new_user.sum() # QA - the row should sum to 1 if we normalised things

    # Now we can predict on this cold-user just like any other
cold_user_preds = model.predict(user_ids = 0,
                                item_ids = [*item_mappings.values()],
                                item_features = item_metadata_list,
                                user_features = scipy.sparse.csr_matrix(new_user))

cold_ranks = np.argsort(-cold_user_preds)[:10]
cold_ranks = pd.DataFrame(zip([*inv_item_mappings.values()], cold_ranks), columns = ["product_name", "rank"])
print(cold_ranks.sort_values(["rank"])[:10])

# We can also use LightFM to generate the embeddings for us by passing our new matrix
cold_user_bias, cold_user_embedding = model.get_user_representations(new_user)
manual_cold_scores = ((cold_user_embedding @ model.get_item_representations(features=item_metadata_list)[1].T + model.get_item_representations(features=item_metadata_list)[0]).T + cold_user_bias).T
np.allclose(manual_cold_scores, cold_user_preds)

   product_name  rank
4        459835     5
6         19278    93
0         10034   679
1        465522   761
3        161949  1314
2         49029  1339
7        395273  1379
5        232172  2208
8         94371  4778
9        414182  8673


False

Трудно сказать, насколько разумными кажутся эти рекомендации, не протестировав их на нескольких пользователях холодного старта, но, по крайней мере, мы видим, что процесс создания матрицы пользователей холодного старта успешно возвращает различные рекомендации для каждого типа пользователей, которых мы создали. Как и в случае с признаками элементов, мы можем заставить LightFM вернуть нам окончательное представление наших пользователей холодного старта, а затем сделать свой собственный точечный продукт для создания прогнозов.

In [131]:
# We can also use LightFM to generate the embeddings for us by passing our new matrix
cold_user_bias, cold_user_embedding = model.get_user_representations(new_user)
manual_cold_scores = ((cold_user_embedding @ model.get_item_representations(features=item_metadata_list)[1].T + model.get_item_representations(features=item_metadata_list)[0]).T + cold_user_bias).T

# XGBoost

In [132]:
# загрузим данные о покупках и иных взаимодействиях items и users
events = pd.read_csv('/kaggle/input/events.csv/events.csv')
# переведем формат времени к обычному виду
events['event_datetime'] = pd.to_datetime(events['timestamp'], unit='ms', origin='unix')
# отсортируем данные от самых ранних записей до поздних
events = events.sort_values(['event_datetime']).reset_index(drop=True)
events.head(5)

,timestamp,visitorid,event,itemid,transactionid,event_datetime
0,1430622004384,693516,addtocart,297662,NaN,2015-05-03 03:00:04.384
1,1430622011289,829044,view,60987,NaN,2015-05-03 03:00:11.289
2,1430622013048,652699,view,252860,NaN,2015-05-03 03:00:13.048
3,1430622024154,1125936,view,33661,NaN,2015-05-03 03:00:24.154
4,1430622026228,693516,view,297662,NaN,2015-05-03 03:00:26.228


In [133]:
print(f'Количество дубликатов до удаления: {len(events)- len(events.drop_duplicates())}')
events.drop_duplicates(inplace=True)
print(f'Количество дубликатов после удаления: {len(events)- len(events.drop_duplicates())}')

Количество дубликатов до удаления: 460
Количество дубликатов после удаления: 0


In [134]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2755641 entries, 0 to 2756100
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   timestamp       int64         
 1   visitorid       int64         
 2   event           object        
 3   itemid          int64         
 4   transactionid   float64       
 5   event_datetime  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 147.2+ MB


## feature engineering

In [135]:
s1 = events[events['event'] == 'transaction']
n = int(s1.shape[0]/2)
s2 = events[events['event'] == 'addtocart'].sample(n=n, random_state = 42)
s3 = events[events['event'] == 'view'].sample(n=n, random_state = 42)
events_c = pd.concat([s1, s2, s3], ignore_index=False)
events_c = events_c[['visitorid', 'event', 'itemid','event_datetime']].astype({'visitorid':'int32','itemid':'int32'}).rename(columns ={'visitorid': 'userid'})

## Adding items feature

In [136]:
# загрузим справочник товаров
item_1 = pd.read_csv('/kaggle/input/item_properties_part1.csv/item_properties_part1.csv')
item_2 = pd.read_csv('/kaggle/input/item_properties_part2.csv/item_properties_part2.csv')
properties_temp = pd.concat([item_1, item_2])
properties_temp.loc[:,'value'] = properties_temp['value'].str.replace('.','',regex=False)

#оставим только те items, которые есть в events, чтобы уменьшить размер данных
list_items_events = list(set(events_c.itemid))
properties = properties_temp[properties_temp['itemid'].isin(list_items_events)]
print(properties_temp.shape)
print(properties.shape)
del s1
del s2
del s3
del item_1
del item_2
gc.collect()

(20275902, 4)
(1394917, 4)


245

In [137]:
len(set(events_c.itemid).difference(set(properties.itemid)))

1341

В данных о транзакциях есть упоминание о items, которых нет в справочнике properties

In [138]:
list_items_events = list(events_c['itemid'].unique())
len(list_items_events)

21415

Нам нужно найти наиболее полезные свойства items.   
А они есть у фактически купленных items.   
По этому их и купили, очевидно.   
Поэтому отфильтруем справочник всех товаров.  
Оставим фактически купленные items.  

Это список наиболее привлекательных items, следовательно свойства этих items наиболее желанны для потребителя.

Преобразуем справочник items. Опишем каждый itemid через список наиболее привлекательных характеристик.

Следующий вариант - соединить свойства и его описание.  
Опиратся просто на свойство я считаю неправильным.   
Обязательно должно быть значение свойства.   
К примеру, молоко. Его свойство - жирность.   
Но значение имеет какая жирность.   
Соединяя зашифрованнные свойства и их описания мы можем прийти к значению - "Жирность 3.2%" или "Цвет-черный".  
А это и есть искомые потребительские свойства.

In [139]:
#приведем данные о свойствах к текстовому виду
properties.loc[:,'value'] = properties['value'].astype("str")
properties.loc[:,'property'] = properties['property'].astype("str")
#Сгруппируем items по itemsid и property. Агрегируем описания свойств
df1 = properties.groupby(['itemid', 'property'], as_index = False)['value'].agg(' '.join)

# Преобразуем данные о свойствах items в удобный для обработки вид.
# Для этого соеденим свойство и его описание для каждого items.
def prop_feature_best(df):
    df_temp =  df.loc[:, ['property','value']].copy()
    len_pf =  df.shape[0]
#     for i in tqdm(range(len_pf)):
    for i in tqdm(list(df.index)):
        list_prop_feat = []
#         print(i)
        split_feat = df_temp.loc[i,'value'].split()
        for f in split_feat:
            list_prop_feat.append(df.loc[i,'property']+'_'+f)
        df_temp.loc[i,'value'] = list_prop_feat
    df_temp.loc[:,'itemid'] = df.loc[:,'itemid']
    df_temp.drop(columns = 'property', inplace=True)
    return df_temp
df_all = prop_feature_best(df1)
df_all

100%|██████████| 589387/589387 [02:37<00:00, 3748.02it/s]


,value,itemid
0,[112_679677],15
1,[159_519769],15
2,[202_789221],15
3,[227_433564],15
4,"[283_433564, 283_245772, 283_789221, 283_80927...",15
...,...,...
589382,"[888_1262739, 888_205682, 888_1050016, 888_126...",466864
589383,[917_205682],466864
589384,[928_1154859],466864
589385,"[available_1, available_1, available_1, availa...",466864


In [140]:
train, test = train_test_split(events_c, test_size=0.3, shuffle=False, random_state=42)

events_deal=train[train['event'] == 'transaction'].groupby(['itemid'])['userid'].agg('count').sort_values(ascending = False)
events_deal[:5]

itemid
461686    133
119736     97
213834     92
312728     46
7943       46
Name: userid, dtype: int64

Найдем 20 наиболее популярных признаков среди фактически купленных items.

In [141]:
# Отсортируем, оставим только купленные items
df_transaction = df_all[df_all['itemid'].isin(list(events_deal.index))]

list_count= []
list_count = list(df_transaction['value'].explode())
print(f'Общее количество характеристик: {len(set(list_count))}')

Общее количество характеристик: 124604


Список 20 самых популярных характеристик у items

In [142]:
cnt = Counter(list_count)
dict_feat = dict(cnt)
most_popular_tags = cnt.most_common(20)
tags = [most_popular_tags[x][0] for x in range(len(most_popular_tags))]
most_popular_tags[:10]

[('available_1', 90104),
 ('available_0', 54516),
 ('888_1284577', 32560),
 ('888_1297729', 16363),
 ('888_350726', 13579),
 ('400_424566', 13554),
 ('888_30603', 13527),
 ('283_30603', 13270),
 ('888_1154859', 13131),
 ('888_832471', 12244)]

Построим матрицу свойств всего справочника items через наиболее популярные признаки

In [143]:
df_all = df_all.groupby(['itemid']).agg('sum')
for x in tags:
    df_all[x] =df_all['value'].apply(lambda y: 1 if str(x) in y else 0)
items_map = df_all.drop(['value'], axis=1).reset_index()
items_map.head(3)

,itemid,available_1,available_0,888_1284577,888_1297729,888_350726,400_424566,888_30603,283_30603,888_1154859,...,112_679677,159_519769,764_1285872,283_150169,400_n720000,888_n36000,888_726612,888_784581,283_1128577,888_1187104
0,15,1,1,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,1,0
1,19,1,1,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,0
2,25,1,1,0,0,0,0,0,1,1,...,1,1,1,0,0,0,0,0,1,0


In [144]:
items_map[items_map.drop(columns = 'itemid').sum(axis=1) == 0]

,itemid,available_1,available_0,888_1284577,888_1297729,888_350726,400_424566,888_30603,283_30603,888_1154859,...,112_679677,159_519769,764_1285872,283_150169,400_n720000,888_n36000,888_726612,888_784581,283_1128577,888_1187104


## Adding users feature

Построение модели рекомендаций с использованием характеристик items и users
Подключим к модели свойства userid.   
Явных свойств users у нас нет. Типа возраста или профессии.   
Поэтому, в качестве таковых, предлагаю использовать время, когда user совершал покупку.   
Преобразуем данные о транзакциях с помощью OneHotEncoder

In [145]:
# считаю целесообразным использовать признаки месяца, дня недели и времени суток покупки
events_c['day_of_week'] = events_c['event_datetime'].map(lambda x: x.weekday())
events_c['Month'] = events_c['event_datetime'].map(lambda x: x.month)
events_c['Hour'] = events_c['event_datetime'].map(lambda x: x.hour)

def get_time_periods(hour):
    if hour >= 3 and hour < 7:
        return 'Dawn'
    elif hour >= 7 and hour < 12:
        return 'Morning'
    elif hour >= 12 and hour < 16:
        return 'Afternoon'
    elif hour >= 16 and hour < 22:
        return 'Evening'
    else:
        return 'Night'

events_c['Day Period'] = events_c['Hour'].map(get_time_periods)
events_c['Day Period'].value_counts()

Day Period
Evening      20254
Night        12145
Dawn          6677
Afternoon     4277
Morning       1560
Name: count, dtype: int64

In [146]:
# one_hot_encoder = OneHotEncoder(dtype=bool)
one_hot_encoder = OneHotEncoder()
columns_to_change = ['day_of_week','Month','Day Period']
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(events_c[columns_to_change]).toarray()
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
encoder_df = pd.DataFrame(data_onehot, columns = column_names)

#merge one-hot encoded columns back with original DataFrame
events_c.reset_index(drop=True, inplace = True)
final_df = events_c.join(encoder_df)
final_df = final_df.drop(columns = ['event_datetime','day_of_week','Month','Hour','Day Period'])
final_df = pd.merge(final_df, items_map, how="outer", on=["itemid", "itemid"])

In [147]:
final_df.loc[:,'event'] = final_df['event'].apply(lambda x: 1 if x == 'transaction' else 0)
final_df.head(3)

,userid,event,itemid,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,...,112_679677,159_519769,764_1285872,283_150169,400_n720000,888_n36000,888_726612,888_784581,283_1128577,888_1187104
0,1124964,1,15,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1124964,0,15,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,325833,1,19,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## train/test split

In [148]:
# Удаляем идентификаторы, оставляем только признаки
X = final_df.drop(columns=['userid', 'itemid', 'event'])
y = final_df['event']

# Разделите данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## XGBoost model

In [149]:
# Настройки для XGBoost с указанием устройства GPU
# params = {
#     'tree_method': 'hist', # Использование GPU для построения деревьев
# #     'objective':'binary:logistic',  # Функция для классификации
#     'objective': 'reg:squarederror',  # Объективная функция для регрессии
#     'device': "cuda",
# }

# # Пример создания DMatrix с указанием устройства GPU
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test)
# # Обучение модели
# bst = xgb.train(params, dtrain)
# # Предсказание
# y_pred = bst.predict(dtest)

model = xgb.XGBRegressor(objective ='reg:squarederror')
model.fit(X_train, y_train)
# Предсказание рейтингов
y_pred = model.predict(X_test)

# Объединим предсказания с исходными данными
results = pd.DataFrame()
results['event'] = final_df.loc[X_test.index, 'event']
results['userid'] = final_df.loc[X_test.index, 'userid']
results['itemid'] = final_df.loc[X_test.index, 'itemid']
results['predicted_rating'] = y_pred

# Функция для расчета Precision@K
def precision_at_k(y_true, y_pred, k):
    idx = np.argsort(y_pred)[::-1][:k]
    relevant_items = sum(y_true[idx] > 0.5)  # Предполагая, что релевантный рейтинг > 0.5
    return relevant_items / k

# Группировка по пользователям
precision_scores = []
for user_id, group in results.groupby('userid'):
    true_ratings = group['event'].values
    predicted_ratings = group['predicted_rating'].values
    precision_scores.append(precision_at_k(true_ratings, predicted_ratings, k=3))  # K=3

# Среднее значение Precision@K
mean_precision_at_k = np.mean(precision_scores)
print(f'Mean Precision@K: {mean_precision_at_k}')

Mean Precision@K: 0.1690307328605201


In [150]:
results.head(3)

,event,userid,itemid,predicted_rating
32420,0,1197758,339479,-0.005738
18181,0,1112398,194972,0.036359
17054,0,842668,183501,0.345930


## Optuna


In [151]:
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

# Определение функции для оптимизации
def objective(trial):
    params = {
#         'device': "cuda",
#         'tree_method': 'hist',  # Для использования GPU
        'objective':'reg:squarederror',  # Функция для регрессии
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True)
    }
# # skikit-learn interface
    model = xgb.XGBRegressor(**params)
    model.fit(X_train_o, y_train_o, eval_set=[(X_test_o, y_test_o)], verbose=False)
    y_pred = model.predict(X_test_o)

#     dtrain = xgb.DMatrix(X_train_o, label=y_train_o)
#     dtest = xgb.DMatrix(X_test_o)
#     # Обучение модели
#     bst = xgb.train(params, dtrain)
#     # Предсказание
#     y_pred = bst.predict(dtest)

    # Объединим предсказания с исходными данными
    results = pd.DataFrame()
    results['event'] = final_df.loc[X_test_o.index, 'event']
    results['userid'] = final_df.loc[X_test_o.index, 'userid']
    results['itemid'] = final_df.loc[X_test_o.index, 'itemid']
    results['predicted_rating'] = y_pred

    # Группировка по пользователям
    precision_scores = []
    for user_id, group in results.groupby('userid'):
        true_ratings = group['event'].values
        predicted_ratings = group['predicted_rating'].values
        precision_scores.append(precision_at_k(true_ratings, predicted_ratings, k=3))  # K = 3

# Среднее значение Precision@K
    mean_precision_at_k = np.mean(precision_scores)

    return - mean_precision_at_k

# Запуск процесса оптимизации
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, timeout=500)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

# Save the best parameters
best_params = study.best_params
for k, v in best_params.items():
    print(k,":",v)

[I 2024-06-29 08:48:55,616] A new study created in memory with name: no-name-605d8472-5540-47e5-8cf9-fef4f8ee74fe
[I 2024-06-29 08:48:58,933] Trial 0 finished with value: -0.166789789050316 and parameters: {'max_depth': 10, 'learning_rate': 0.00766208462068206, 'min_child_weight': 1, 'subsample': 0.7209184240587241, 'colsample_bytree': 0.892568354168747, 'reg_alpha': 0.48260386241695014, 'reg_lambda': 0.003826189220341985}. Best is trial 0 with value: -0.166789789050316.
[I 2024-06-29 08:49:01,918] Trial 1 finished with value: -0.16715915620126404 and parameters: {'max_depth': 8, 'learning_rate': 0.025104821961032933, 'min_child_weight': 7, 'subsample': 0.5119769492355749, 'colsample_bytree': 0.6149888829164998, 'reg_alpha': 0.0033439510400396243, 'reg_lambda': 7.231041815874082e-05}. Best is trial 1 with value: -0.16715915620126404.
[I 2024-06-29 08:49:05,192] Trial 2 finished with value: -0.16695395222851514 and parameters: {'max_depth': 10, 'learning_rate': 0.010128857401103381, 'mi

Number of finished trials: 20
Best trial: {'max_depth': 3, 'learning_rate': 0.002402076360201326, 'min_child_weight': 3, 'subsample': 0.8543414136375534, 'colsample_bytree': 0.9955190743206662, 'reg_alpha': 6.121615210463039e-06, 'reg_lambda': 0.6355532500884982}
max_depth : 3
learning_rate : 0.002402076360201326
min_child_weight : 3
subsample : 0.8543414136375534
colsample_bytree : 0.9955190743206662
reg_alpha : 6.121615210463039e-06
reg_lambda : 0.6355532500884982


In [152]:
# skikit-learn interface
model = xgb.XGBRegressor(**best_params, random_state=123)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# # Обучение модели
# bst = xgb.train(params, dtrain)
# y_pred = bst.predict(dtest)

# Объединим предсказания с исходными данными
results = pd.DataFrame()
results['y_actual'] = final_df.loc[X_test.index, 'event']
results['userid'] = final_df.loc[X_test.index, 'userid']
results['itemid'] = final_df.loc[X_test.index, 'itemid']
results['y_recommended'] = y_pred

# Группировка по пользователям
precision_scores = []
for user_id, group in results.groupby('userid'):
    true_ratings = group['y_actual'].values
    predicted_ratings = group['y_recommended'].values
    precision_scores.append(precision_at_k(true_ratings, predicted_ratings, k=3))  # K = 5 как пример

# Среднее значение Precision@K
mean_precision_at_k = np.mean(precision_scores)
print(f'Mean Precision@K: {mean_precision_at_k}')

Mean Precision@K: 0.16962174940898345


In [153]:
# from google.colab import userdata
# import os

# os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
# os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [154]:
# ! pip install kaggle -q

In [155]:
# ! kaggle kernels output pavelnovikov888/notebook525ca83e7f -p /content/drive/MyDrive/f_project

In [156]:
# ! tar -xzf "/content/drive/MyDrive/f_project/algo_user_arch.tar.gz"

In [157]:
# ! cp /content/kaggle/working/algo_user /content/drive/MyDrive/f_project

In [158]:
# Десериализация
# with open ('/content/kaggle/working/algo_user', 'rb') as fp:
# algo_user = pickle.load(fp)